### Newsbot to write a daily AI news summary using langgraph
- Save a list of HTML files from sources.yaml (tech news sites)
- Extract URLs for the news stories
- Filter URLs to remove duplicates, articles seen before, and non-AI articles (using a ChatGPT prompt)
- Perform headline topic analysis and sort by topic to help the AI structure the response by topic
- Scrape and summarize individual articles
- Compose and email the summary


In [ ]:
# to selectively re-import as needed
# import sys
# del sys.modules['ainb_llm']
# del sys.modules['ainb_const']
# del sys.modules['ainb_utilities']
# del sys.modules['ainb_webscrape']


In [1]:
from datetime import datetime
import os
import yaml
import sqlite3
import json
from collections import Counter
import uuid
from typing import TypedDict, Annotated

import operator
import pickle

import langchain
from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_core.prompts import (ChatPromptTemplate, MessagesPlaceholder, PromptTemplate,
                                    SystemMessagePromptTemplate, HumanMessagePromptTemplate)
# from langchain.agents import create_tool_calling_agent, AgentExecutor
# from langchain.memory import ChatMessageHistory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
# from langchain.tools import BaseTool, StructuredTool, tool
# from langchain_community.chat_models import ChatOllama
# from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import SimpleJsonOutputParser, JsonOutputParser, StrOutputParser
# from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
# from langchain_core.outputs import LLMResult

from langgraph.graph import StateGraph, START, END

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt

import sklearn
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

import bs4
# from bs4 import BeautifulSoup
# import requests
from urllib.parse import urljoin, urlparse

import multiprocessing
# from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText

import openai
from openai import OpenAI

import trafilatura

VERBOSE=1
from ainb_const import (DOWNLOAD_DIR, PAGES_DIR,
                        MODEL, LOWCOST_MODEL, HIGHCOST_MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE, SQLITE_DB,
                        HOSTNAME_SKIPLIST, SITE_NAME_SKIPLIST,
                       )
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article,
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file,
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory,
                            process_url_queue_factory, get_google_news_redirects)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries,
                      fetch_openai_summary, count_tokens, trunc_tokens, 
                      categorize_headline, categorize_df, clean_topics,
                      get_site_name, fetch_missing_site_names)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


In [2]:
print(f"LangChain         {langchain.__version__}")
print(f"OpenAI            {openai.__version__}")
# print(f"smtplib           {smtplib.sys.version}")
print(f"trafilatura       {trafilatura.__version__}")
# print(f"bs4               {bs4.__version__}")
print(f"numpy             {np.__version__}")
print(f"pandas            {pd.__version__}")
print(f"sklearn           {sklearn.__version__}")
print(f"umap              {umap.__version__}")


LangChain         0.3.0
OpenAI            1.47.0
trafilatura       1.12.2
numpy             1.26.4
pandas            2.2.3
sklearn           1.5.2
umap              0.5.6


In [3]:
# a basic LLM call with langchain
model = ChatOpenAI(model=MODEL)

model.invoke([
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content='Listen to me. You are beautiful. You are perfect and I love you.'),
])


2024-09-27 08:06:21,111 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Ascoltami. Sei bellissima. Sei perfetta e ti amo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 34, 'total_tokens': 50, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'chatgpt-4o-latest', 'system_fingerprint': 'fp_2bd44569e0', 'finish_reason': 'stop', 'logprobs': None}, id='run-99ebad94-fd19-4d60-860f-f15e0b684347-0', usage_metadata={'input_tokens': 34, 'output_tokens': 16, 'total_tokens': 50})

In [4]:
# use template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
parser = StrOutputParser()
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "hi"})


2024-09-27 08:06:25,969 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Ciao'

In [5]:
prompt_inputs = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

system_template = 'Translate the following into {language}:'
user_template = 'Listen to me. You are {adjective1}. You are {adjective2} and I love you.'

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

parser = StrOutputParser()

chain = prompt_template | model | parser

start_time = datetime.now()
for tpl in prompt_inputs:
    response = ""
    # stream tokens as they are generated
    for r in chain.stream(tpl):
        print(r, end="")
        response += r
end_time = datetime.now()

difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")


2024-09-27 08:06:27,011 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.

2024-09-27 08:06:27,486 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.

2024-09-27 08:06:28,427 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Escúchame. Eres perfecta. Eres hermosa y te amo.

2024-09-27 08:06:29,562 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Ascoltami. Sei straordinario. Sei magnifico e ti amo.

2024-09-27 08:06:30,230 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Figyelj rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.

Elapsed seconds: 3.749454


In [6]:
# same but send all at once using asyncio

async def async_langchain(chain, input_dict):
    response = await chain.ainvoke(input_dict)
    return response


prompt_templates = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

start_time = datetime.now()
tasks = []
for d in prompt_templates:
    task = asyncio.create_task(async_langchain(chain, d))
    tasks.append(task)
responses = await asyncio.gather(*tasks)
end_time = datetime.now()


difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")
print("\n".join(responses))


2024-09-27 08:06:31,204 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-27 08:06:31,208 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-27 08:06:31,211 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-27 08:06:31,308 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-27 08:06:31,954 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




Elapsed seconds: 1.459341
Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.
Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.
Escúchame. Eres perfecto/a. Eres hermoso/a y te amo.
Ascoltami. Sei straordinario. Sei magnifico e ti amo.
Figyelj rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.


In [7]:
DEBUG = True
N_BROWSERS = 4
# before_date="2024-09-23 10:00:00"
before_date = None


In [8]:
# class to maintain settings and state within graph

class AgentState(TypedDict):
    AIdf: dict                #  the current working set of headlines (pandas dataframe not supported)
    before_date: str          #  ignore stories before this date for deduplication (force reprocess since)
    do_download: bool         #  if False use existing files, else download from sources
    sources: dict             #  sources to scrap
    sources_reverse: dict     #  map file names to sources
    headline_str: str         #  headline text email
    bullets: str              #  bullet point summary email
    summary: str              #  final summary
    n_edits: int              #  count edit iterations so we don't keep editing forever
    edit_complete: bool       #  edit will update if no more edits to make
    # message thread with OpenAI
    messages: Annotated[list[AnyMessage], operator.add]


In [9]:
# Initialize reading configurations from YAML file

def fn_initialize(state: AgentState) -> AgentState:
    """
    Initializes the agent state by loading source configurations from SOURCECONFIG (sources.yaml) .

    Args:
        state (AgentState): The current state of the agent.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        AgentState: The updated state of the agent.

    Raises:
        yaml.YAMLError: If there is an error while loading the YAML file.

    """

    #  load sources to scrape from sources.yaml
    with open(SOURCECONFIG, "r") as stream:
        try:
            state['sources'] = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

    log(f"Initialized {len(state['sources'])} items in sources from {SOURCECONFIG}")

    # make a reverse dict to map file titles to source names
    state['sources_reverse'] = {}
    for k, v in state['sources'].items():
        log(f"{k} -> {v['url']} -> {v['title']}.html")
        v['sourcename'] = k
        # map filename (title) to source name
        state['sources_reverse'][v['title']] = k

    log(f"Initialized {len(state['sources_reverse'])} items in sources_reverse")

    return state


if DEBUG:
    test_state = AgentState()
    fn_initialize(test_state)


2024-09-27 08:06:36,312 - AInewsbot - INFO - Initialized 17 items in sources from sources.yaml
2024-09-27 08:06:36,312 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-09-27 08:06:36,313 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology.html
2024-09-27 08:06:36,313 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Business Insider Tech.html
2024-09-27 08:06:36,313 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> FT Technology.html
2024-09-27 08:06:36,313 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Feedly AI.html
2024-09-27 08:06:36,313 - AInewsbot -

In [10]:
# scrape sources with selenium and save local files in DOWNLOAD_DIR (htmldata)
def fn_download_sources(state: AgentState) -> AgentState:
    """
    Scrapes sources and saves HTML files.
    If state["do_download"] is True, deletes all files in DOWNLOAD_DIR (htmldata) and scrapes fresh copies.
    If state["do_download"] is False, uses existing files in DOWNLOAD_DIR.
    Uses state["sources"] for config info on sources to scrape
    For each source, saves the current filename to state["sources"][sourcename]['latest']

    Args:
        state (AgentState): The current state of the agent.
        do_delete (bool, optional): Whether to delete files in DOWNLOAD_DIR. Defaults to True.

    Returns:
        AgentState: The updated state of the agent.
    """

    if state["do_download"]:
        # empty download directory
        delete_files(DOWNLOAD_DIR)

        # save each file specified from sources
        log(f"Saving HTML files using {N_BROWSERS} browsers")

        # Create a queue for multiprocessing and populate it
        queue = multiprocessing.Queue()
        for item in state["sources"].values():
            queue.put(item)

        # Function to take the queue and pop entries off and process until none are left
        # lets you create an array of functions with different args
        callable = process_source_queue_factory(queue)

        saved_pages = launch_drivers(N_BROWSERS, callable)
        for sourcename, file in saved_pages:
            log(f"Downloaded {sourcename} to {file}")
            state['sources'][sourcename]['latest'] = file
        log(f"Saved {len(saved_pages)} HTML files")

    else:   # don't delete, just get list of existing files
        log(f"Web fetch disabled, using existing files in {DOWNLOAD_DIR}")
        # Get the current date
        datestr = datetime.now().strftime("%m_%d_%Y")
        files = [os.path.join(DOWNLOAD_DIR, file)
                 for file in os.listdir(DOWNLOAD_DIR)]
        # filter files with today's date ending in .html
        files = [
            file for file in files if datestr in file and file.endswith(".html")]
        log(f"Found {len(files)} previously downloaded files")
        for file in files:
            log(file)

        saved_pages = []
        for file in files:
            filename = os.path.basename(file)
            # locate date like '01_14_2024' in filename
            position = filename.find(" (" + datestr)
            basename = filename[:position]
            # match to source name
            sourcename = state["sources_reverse"].get(basename)
            if sourcename is None:
                log(f"Skipping {basename}, no sourcename metadata")
                continue
            state["sources"][sourcename]['latest'] = file

    return state

if DEBUG:
    test_state["do_download"] = True
    _ = fn_download_sources(test_state)


2024-09-27 08:06:37,998 - AInewsbot - INFO - Saving HTML files using 4 browsers
2024-09-27 08:06:38,025 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:06:38,027 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:06:38,027 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:06:38,028 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:06:55,361 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:06:55,361 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:06:55,362 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:06:55,362 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:06:55,362 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-09-27 08:06:55,363 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-09-27 08:06:55,363 - AInewsbot - INFO -

2024-09-27 08:08:28,634 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (09_27_2024 08_08_28 AM).html as utf-8
2024-09-27 08:08:28,637 - AInewsbot - INFO - Processing Reddit
2024-09-27 08:08:28,637 - AInewsbot - INFO - get_files(Reddit multiple subreddits) - starting get_files https://www.reddit.com/r/ChatGPT+ChatGPTCoding+MacOS+MachineLearning+OpenAI+ProgrammerHumor+Windows10+battlestations+buildapc+cordcutters+dataisbeautiful+gadgets+hardware+linux+msp+programming+realtech+software+talesfromtechsupport+tech+technews+technology+techsupportgore+windows/top/?sort=top&t=day
2024-09-27 08:08:29,161 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteErro

2024-09-27 08:09:27,293 - AInewsbot - INFO - get_files(Feedly AI) - Saving Feedly AI (09_27_2024 08_09_27 AM).html as utf-8
2024-09-27 08:09:27,296 - AInewsbot - INFO - Quit webdriver
2024-09-27 08:09:29,540 - AInewsbot - INFO - returned 17
2024-09-27 08:09:29,541 - AInewsbot - INFO - Downloaded Business Insider to htmldata/Business Insider Tech (09_27_2024 08_08_06 AM).html
2024-09-27 08:09:29,541 - AInewsbot - INFO - Downloaded Google News to htmldata/Google News - Technology - Artificial intelligence (09_27_2024 08_08_28 AM).html
2024-09-27 08:09:29,541 - AInewsbot - INFO - Downloaded Reddit to htmldata/Reddit multiple subreddits (09_27_2024 08_09_00 AM).html
2024-09-27 08:09:29,542 - AInewsbot - INFO - Downloaded Bloomberg Tech to htmldata/Bloomberg Technology (09_27_2024 08_08_07 AM).html
2024-09-27 08:09:29,542 - AInewsbot - INFO - Downloaded Hacker News to htmldata/Hacker News Page 1 (09_27_2024 08_08_18 AM).html
2024-09-27 08:09:29,542 - AInewsbot - INFO - Downloaded HackerNoon

In [11]:
def fn_extract_urls(state: AgentState) -> AgentState:
    """
    Extracts news URLs from the latest HTML files matching the patterns defined in the state['sources'] configuration info.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the extracted URLs stored in state['AIdf'].
    """
    # Parse news URLs and titles from downloaded HTML files
    log("Parsing html files")
    all_urls = []
    for sourcename, sourcedict in state['sources'].items():
        filename = sourcedict.get('latest')
        if not filename:
            log(f"no filename found for {sourcename}")
            continue

        log(sourcename + ' -> ' + filename)
        links = parse_file(state['sources'][sourcename])
        log(f"{len(links)} links found")
        all_urls.extend(links)

    log(f"Saved {len(all_urls)} links")

    # make a pandas dataframe of all the links found
    AIdf = (
        pd.DataFrame(all_urls)
        .groupby("url")
        .first()
        .reset_index()
        .sort_values("src")[["src", "title", "url"]]
        .reset_index(drop=True)
        .reset_index(drop=False)
        .rename(columns={"index": "id"})
    )
    state['AIdf'] = AIdf.to_dict()

    return state


if DEBUG:
    _ = fn_extract_urls(test_state)



2024-09-27 08:09:29,563 - AInewsbot - INFO - Parsing html files
2024-09-27 08:09:29,565 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (09_27_2024 08_08_06 AM).html
2024-09-27 08:09:29,599 - AInewsbot - INFO - parse_file - found 252 raw links
2024-09-27 08:09:29,602 - AInewsbot - INFO - parse_file - found 27 filtered links
2024-09-27 08:09:29,602 - AInewsbot - INFO - 27 links found
2024-09-27 08:09:29,602 - AInewsbot - INFO - Bloomberg Tech -> htmldata/Bloomberg Technology (09_27_2024 08_08_07 AM).html
2024-09-27 08:09:29,635 - AInewsbot - INFO - parse_file - found 210 raw links
2024-09-27 08:09:29,638 - AInewsbot - INFO - parse_file - found 27 filtered links
2024-09-27 08:09:29,638 - AInewsbot - INFO - 27 links found
2024-09-27 08:09:29,638 - AInewsbot - INFO - Business Insider -> htmldata/Business Insider Tech (09_27_2024 08_08_06 AM).html
2024-09-27 08:09:29,665 - AInewsbot - INFO - parse_file - found 309 raw links
2024-09-27 08:09:29,669 - AInewsbot - INFO - parse_file 

In [12]:
# s/b 17 but if some like bloomberg or wsj are missing, maybe got a robot block
# in that case download missing ones manually and then re-run extract
print(len(pd.DataFrame(test_state["AIdf"]).groupby('src').count()))
pd.DataFrame(test_state["AIdf"]).groupby('src').count()


17


,id,title,url
src,,,
Ars Technica,27,27,27
Bloomberg Tech,22,22,22
Business Insider,50,50,50
FT Tech,61,61,61
Feedly AI,66,66,66
Google News,57,57,57
Hacker News,26,26,26
Hacker News 2,26,26,26
HackerNoon,97,97,97


In [13]:
# sqlite schema to store previously processed 
# CREATE TABLE news_articles (
#     id INTEGER PRIMARY KEY,
#     src TEXT,
#     title TEXT,
#     url TEXT UNIQUE,
#     isAI BOOLEAN,
#     article_date DATE
# , timestamp DATETIME, actual_url TEXT, actual_src TEXT);
#
# CREATE TABLE sites (
#     id INTEGER PRIMARY KEY AUTOINCREMENT,
#     hostname TEXT UNIQUE NOT NULL,
#     site_name TEXT NOT NULL
# );



In [14]:
# filter and clean URLs for new AI stories

def fn_filter_urls(state: AgentState) -> AgentState:
    """
    Filters the URLs in state["AIdf"] to include only those that have not been previously seen,
    and are related to AI according to the response from a ChatGPT prompt.

    Args:
        state (AgentState): The current state of the agent.
        before_date (str, optional): The date before which the URLs should be filtered. Defaults to "".

    Returns:


        AgentState: The updated state of the agent with the filtered URLs stored in state["AIdf"].

    """
    # filter to URL not previously seen
    AIdf = pd.DataFrame(state["AIdf"])

    AIdf = filter_unseen_urls_db(AIdf, before_date=state["before_date"])

    if len(AIdf) == 0:
        log("No new URLs, returning")
        return state

    # dedupe identical headlines
    # filter similar titles differing by type of quote or something
    AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)
    AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
    AIdf = AIdf.sort_values("src") \
        .groupby("title_clean") \
        .first() \
        .reset_index(drop=True) \
        .drop(columns=['id']) \
        .reset_index() \
        .rename(columns={'index': 'id'})
    log(f"Found {len(AIdf)} unique AI headlines")

    # structured response format
    json_schema = {
        "name": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "isai_array": {
                    "type": "array",
                    "items": {
                        "type": "object",
                                "properties": {
                                    "id": {
                                        "type": "number"
                                    },
                                    "isAI": {
                                        "type": "boolean"
                                    }
                                },
                        "required": ["id", "isAI"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["isai_array"],
            "additionalProperties": False
        }
    }

    # filter AI-related headlines using a prompt
    pages = paginate_df(AIdf)
    enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT, json_schema=json_schema))
    filter_df = pd.DataFrame(enriched_urls)
    
    try:  # for idempotency
        AIdf = AIdf.drop(columns=['isAI'])
    except:
        pass

    # merge returned df with isAI column into original df on id column
    AIdf = pd.merge(AIdf, filter_df, on="id", how="outer")
    log(AIdf.columns)
    # set hostname based on actualurl
    AIdf['actual_url'] = AIdf['url']    # ideally resolve redirects but Google News blocks
    AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)

    # update SQLite database with all seen URLs (we are doing this using url and ignoring redirects)
    log(f"Inserting {len(AIdf)} URLs into {SQLITE_DB}")
    conn = sqlite3.connect(SQLITE_DB)
    cursor = conn.cursor()
    for row in AIdf.itertuples():
        insert_article(conn, cursor, row.src, row.hostname, row.title,
                       row.url, row.actual_url, row.isAI, datetime.now().date())

    # keep headlines that are related to AI
    AIdf = AIdf.loc[AIdf["isAI"]==1] \
        .reset_index(drop=True)  \
        .reset_index()  \
        .drop(columns=["id"])  \
        .rename(columns={'index': 'id'})

    log(f"Found {len(AIdf)} AI headlines")

    # update actual URLs for Google News redirects
    # I think Google changed something so this no longer works, instead of a 301 redirct
    # get a javascript page that redirects. Also tomorrow we might see different URLs for same stories
    # AIdf = get_google_news_redirects(AIdf)

    conn = sqlite3.connect('articles.db')
    query = "select * from sites"
    sites_df = pd.read_sql_query(query, conn)
    sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}
    conn.close()

    # get clean site_name
    AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))

    # if any missing clean site names, populate them using OpenAI
    missing_site_names =  len(AIdf.loc[AIdf['site_name']==""])
    if missing_site_names:
        log(f"Asking OpenAI for {missing_site_names} missing site names")
        responses = asyncio.run(fetch_missing_site_names(AIdf))
        # update site_dict from responses
        new_urls = []
        for r in responses:
            if r['url'].startswith('https://'):
                r['url'] = r['url'][8:]
            new_urls.append(r['url'])
            sites_dict[r['url']] = r['site_name']
            log(f"Looked up {r['url']} -> {r['site_name']}")
        # update sites table with new names
        for url in new_urls:
            sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
            log(f"Updated {url} -> {sites_dict[url]}")
            conn.execute(sqlstr, (url, sites_dict[url]))
            conn.commit()
        # reapply to AIdf with updated sites
        AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))
    else:
        log("No missing site names")
  
    # drop banned slop sites

    AIdf = AIdf.loc[~AIdf["hostname"].str.lower().isin(HOSTNAME_SKIPLIST)]
    AIdf = AIdf.loc[~AIdf["site_name"].str.lower().isin(SITE_NAME_SKIPLIST)]
    
    state["AIdf"] = AIdf.to_dict()
    return state


if DEBUG:
#     before_date="2024-09-23 10:00:00"
    before_date = None
    test_state["before_date"] = before_date
#     test_state["before_date"] = None
    _ = fn_filter_urls(test_state)


2024-09-27 08:09:43,215 - AInewsbot - INFO - Querying SQLite with where_clause: 
2024-09-27 08:09:43,834 - AInewsbot - INFO - URLs in orig_df: 939
2024-09-27 08:09:43,836 - AInewsbot - INFO - Existing URLs in DB: 151890
2024-09-27 08:09:43,860 - AInewsbot - INFO - New URLs in df filtered by URL: 268
2024-09-27 08:09:43,943 - AInewsbot - INFO - Existing src+title: 13
2024-09-27 08:09:43,944 - AInewsbot - INFO - New URLs in df filtered by src+title: 255
2024-09-27 08:09:43,955 - AInewsbot - INFO - Found 252 unique AI headlines
2024-09-27 08:09:44,196 - AInewsbot - INFO - Applying prompt to 6 pages using gpt-4o-mini
2024-09-27 08:09:44,197 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:09:44,198 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:09:44,199 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:09:44,199 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:09:44,200 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:09:44,200 - AInewsbot - INFO - sent 2 items 
2024-09-27 08:09:45,

In [15]:
print(pd.DataFrame(test_state['AIdf']).columns)
len(pd.DataFrame(test_state['AIdf']))


Index(['id', 'src', 'title', 'url', 'isAI', 'actual_url', 'hostname',
       'site_name'],
      dtype='object')


98

In [16]:
# Function for topic extraction

def clean_topics(row, lcategories):
    """
    Cleans the extracted_topics and assigned_topics by removing certain common topics and combining them into a single list.

    Args:
        row (pandas.Series): The row containing the extracted_topics and assigned_topics.
        lcategories (set): The set of lowercase categories.

    Returns:
        list: The cleaned and combined list of topics.
    """
    extracted_topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(extracted_topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]

    return combined


async def do_cat(AIdf, categories):
    """
    Sends a prompt to ChatGPT to select topics for the title for each row in AIdf
    which match the topics in categories.

    Args:
        AIdf (pandas.DataFrame): The DataFrame containing the headlines.
        categories (list): The list of topics to match with the headlines.

    Returns:
        dict: A dictionary where the keys are the row IDs and the values are lists
        of selected topics for each headline.

    """

    catdict = {}
    async with aiohttp.ClientSession() as session:
        for i, row in enumerate(AIdf.itertuples()):
            tasks = []
            log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
            h = row.title
            log(h)
            for c in categories:
                task = asyncio.create_task(categorize_headline(h, c, session))
                tasks.append(task)
            responses = await asyncio.gather(*tasks)
            catdict[row.id] = [item for sublist in responses for item in sublist]
            log(str(catdict[row.id]))

    return catdict


def fn_topic_analysis(state: AgentState) -> AgentState:

    """
    Extracts and selects topics for each headline in the state['AIdf'] dataframe, scrubs them, and stores them back in the dataframe.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the extracted and selected topics stored in state['AIdf'].
    """
    AIdf = pd.DataFrame(state["AIdf"])
    pages = paginate_df(AIdf)
    # apply topic extraction prompt to AI headlines
    log("start free-form topic extraction")
    json_schema = {
        "name": "extracted_topics",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "extracted_topics": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "id": {
                                "type": "number",
                            },
                            "topics": {
                                "type": "array",
                                "items": {
                                    "type": "string",
                                },
                            },
                        },
                        "required": ["id", "topics"],
                        "additionalProperties": False,
                    }
                }
            },
            "required": ["extracted_topics"],
            "additionalProperties": False,
        }
    }
    response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT, json_schema=json_schema))
    topic_df = pd.DataFrame(response)
    topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})

    log(f"{len(topic_df)} free-form topics extracted")
    all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
    item_counts = Counter(all_topics)
    filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
    categories = sorted(CANONICAL_TOPICS)
    # use categories that are canonical or show up twice in freeform
    lcategories = set([c.lower() for c in categories] + [c.lower() for c in filtered_topics])
    # new topics
    log([c for c in filtered_topics if c not in categories])

    catdict = asyncio.run(categorize_headline(AIdf, categories=categories))
    topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))

    topic_df["topics"] = topic_df.apply(lambda t: clean_topics(t, lcategories), axis=1)
    topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)

    try: # for idempotency
        AIdf = AIdf.drop(columns=['topic_str', 'title_topic_str'])
    except:
        pass

    AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
    AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)

    state["AIdf"] = AIdf.to_dict()
    return state

if DEBUG:
    _ = fn_topic_analysis(test_state)


2024-09-27 08:10:11,257 - AInewsbot - INFO - start free-form topic extraction
2024-09-27 08:10:11,257 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:10:11,258 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:10:11,258 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:10:22,433 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:10:25,544 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:10:25,547 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:10:25,549 - AInewsbot - INFO - 98 free-form topics extracted
2024-09-27 08:10:25,553 - AInewsbot - INFO - ['generative ai', 'trade', 'nvidia', 'openai', 'sam altman', 'amazon', 'anthropic', 'apple', 'investors', 'eu', 'google', 'productivity', 'social media', 'chatgpt', 'voice mode', 'regulation', 'llm', 'hugging face', 'ftc', 'gemini', 'ai talent', 'meta', 'cfo', 'funding', 'content creation', 'risks', 'samsung', 'for-profit', 'investment', 'issues']
2024-09-27 08:10:25,554 - AInewsbot

2024-09-27 08:12:21,575 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:12:21,809 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:12:21,812 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:12:21,813 - AInewsbot - INFO - Bill Gates, topic 16 of 154
2024-09-27 08:12:21,814 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:12:21,814 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:12:21,818 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:12:27,338 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:12:29,540 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:12:29,542 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:12:29,542 - AInewsbot - INFO - Books & Publishing, topic 17 of 154
2024-09-27 08:12:29,544 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:12:29,546 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:12:29,547 - AInewsbot - INFO - sent 48 items 
2024-09-27 0

2024-09-27 08:14:27,408 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:14:27,409 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:14:33,308 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:14:33,882 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:14:33,884 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:14:33,885 - AInewsbot - INFO - Customer service, topic 33 of 154
2024-09-27 08:14:33,886 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:14:33,887 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:14:33,888 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:14:42,050 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:14:42,293 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:14:42,296 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:14:42,296 - AInewsbot - INFO - Cybersecurity, topic 34 of 154
2024-09-27 08:14:42,297 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 

2024-09-27 08:16:36,059 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:16:36,060 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:16:36,061 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:16:41,332 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:16:42,336 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:16:42,338 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:16:42,339 - AInewsbot - INFO - Facial Recognition, topic 50 of 154
2024-09-27 08:16:42,340 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:16:42,341 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:16:42,342 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:16:54,825 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:17:00,141 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:17:00,143 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:17:00,144 - AInewsbot - INFO - Fashion, topic 51 of 154
2024-09-27 08:1

2024-09-27 08:18:52,380 - AInewsbot - INFO - Hollywood, topic 66 of 154
2024-09-27 08:18:52,381 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:18:52,383 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:18:52,385 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:18:57,268 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:18:57,440 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:18:57,443 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:18:57,444 - AInewsbot - INFO - Hugging Face, topic 67 of 154
2024-09-27 08:18:57,445 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:18:57,447 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:18:57,448 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:19:06,546 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:19:15,453 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:19:15,460 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:19:15

2024-09-27 08:21:20,264 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:21:20,268 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:21:20,268 - AInewsbot - INFO - Legal issues, topic 83 of 154
2024-09-27 08:21:20,270 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:21:20,271 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:21:20,272 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:21:26,320 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:21:26,322 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:21:26,324 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:21:26,325 - AInewsbot - INFO - Lifestyle & Travel, topic 84 of 154
2024-09-27 08:21:26,326 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:21:26,327 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:21:26,328 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:21:33,394 - AInewsbot - INFO - got dict with 48 items 
2024-09-27

2024-09-27 08:23:38,952 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:23:45,651 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:23:45,833 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:23:45,835 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:23:45,836 - AInewsbot - INFO - Opinion, topic 100 of 154
2024-09-27 08:23:45,837 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:23:45,838 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:23:45,839 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:23:50,792 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:23:51,260 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:23:51,261 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:23:51,263 - AInewsbot - INFO - Oracle, topic 101 of 154
2024-09-27 08:23:51,267 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:23:51,269 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:23:51,270 -

2024-09-27 08:25:32,923 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:25:32,924 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:25:32,925 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:25:38,553 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:25:38,964 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:25:38,966 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:25:38,967 - AInewsbot - INFO - Russia, topic 117 of 154
2024-09-27 08:25:38,968 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:25:38,969 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:25:38,970 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:25:45,420 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:25:45,579 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:25:45,582 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:25:45,583 - AInewsbot - INFO - SAP, topic 118 of 154
2024-09-27 08:25:45,584 - AIn

2024-09-27 08:27:42,775 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:27:42,776 - AInewsbot - INFO - Stability AI, topic 133 of 154
2024-09-27 08:27:42,776 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:27:42,778 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:27:42,781 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:27:54,301 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:27:58,742 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:27:58,743 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:27:58,744 - AInewsbot - INFO - Stocks, topic 134 of 154
2024-09-27 08:27:58,745 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:27:58,746 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:27:58,747 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:28:05,183 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:28:05,912 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:28:05,

2024-09-27 08:30:08,506 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:30:09,621 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:30:09,623 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:30:09,624 - AInewsbot - INFO - Virtual & Augmented Reality, topic 150 of 154
2024-09-27 08:30:09,624 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:30:09,625 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:30:09,627 - AInewsbot - INFO - sent 48 items 
2024-09-27 08:30:15,765 - AInewsbot - INFO - got dict with 48 items 
2024-09-27 08:30:17,336 - AInewsbot - INFO - got dict with 50 items 
2024-09-27 08:30:17,340 - AInewsbot - INFO - Processed 98 responses.
2024-09-27 08:30:17,341 - AInewsbot - INFO - Virtual Assistants, topic 151 of 154
2024-09-27 08:30:17,342 - AInewsbot - INFO - Applying prompt to 2 pages using gpt-4o-mini
2024-09-27 08:30:17,343 - AInewsbot - INFO - sent 50 items 
2024-09-27 08:30:17,344 - AInewsbot - INFO - sent 48 

In [17]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):
    """
    Generates a name for a cluster based on a list of headline topics.

    Parameters:
    session (aiohttp.ClientSession): The client session for making async HTTP requests.
    topic_list_str (str): A string containing the list of headline topics.
    max_retries (int, optional): The maximum number of retries in case of an error. Defaults to 3.
    model (str, optional): The model to use for generating the topic name. Defaults to LOWCOST_MODEL.

    Returns:
    dict: A dictionary containing the generated topic name.

    Example Usage:
    title_topic_str_list = "Headline 1 (Topic: Topic 1)\n\nHeadline 2 (Topic: Topic 2)"
    result = await write_topic_name(session, title_topic_str_list)
    print(result)

    Output:
    {"topic_title": "Generated Topic Name"}
    ```
    """
    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses.
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
#             print(topic_list_str)
            response = asyncio.run(fetch_openai(session, payload))
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}


async def afn_topic_clusters(state: AgentState) -> AgentState:
    """
    Fetches embeddings for the headlines, creates clusters of similar articles using DBSCAN, and sorts
    using the clusters and a traveling salesman shortest traversal in embedding space.

    Parameters:
    state (AgentState): The state of the agent.

    Returns:
    AgentState: The updated state of the agent.

    """
    AIdf = pd.DataFrame(state["AIdf"])

    log(f"Fetching embeddings for {len(AIdf)} headlines")
    embedding_model = 'text-embedding-3-large'
    client = OpenAI()
    response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                        model=embedding_model)
    embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

    # greedy traveling salesman sort
    log(f"Sort with nearest_neighbor_sort sort")
    sorted_indices = nearest_neighbor_sort(embedding_df)
    AIdf['sort_order'] = sorted_indices

    # do dimensionality reduction on embedding_df and cluster analysis
    log(f"Perform dimensionality reduction")
    with open("reducer.pkl", 'rb') as file:
        # Load the model from the file
        reducer = pickle.load(file)
    reduced_data = reducer.transform(embedding_df)
    log(f"Cluster with DBSCAN")
    dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
    AIdf['cluster'] = dbscan.fit_predict(reduced_data)
    AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999

    # sort first by clusters found by DBSCAN, then by semantic ordering
    AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
        .reset_index(drop=True) \
        .reset_index() \
        .drop(columns=["id"]) \
        .rename(columns={'index': 'id'})

    # show clusters
    cluster_topics = []
    with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
        async with aiohttp.ClientSession() as session:
            for i in range(30):
                tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
                if len(tmpdf) ==0:
                    break
                display(tmpdf)
                title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
                cluster_topic = await write_topic_name(session, title_topic_str_list)
                cluster_topic = cluster_topic['topic_title']
                cluster_topics.append(cluster_topic)
                log(f"I dub this cluster: {cluster_topic}")
    state["cluster_topics"] = cluster_topics
    AIdf["cluster_name"] = AIdf['cluster'].apply(lambda i: cluster_topics[i] if i < len(cluster_topics) else "")
    state["AIdf"] = AIdf.to_dict()
    return state

# TODO: could add a quality rating for stories based on site reputation, length, complexity of story
# could then add the quality rating to the summaries and tell the prompt to favor high-quality stories
# could put summaries into vector store and retrieve stories by topic. but then you will have to deal
# with duplicates across categories, ask the prompt to dedupe

def fn_topic_clusters(state: AgentState) -> AgentState:
    "call async afn_topic_clusters on state"
    asyncio.run(afn_topic_clusters(state))
    return state


if DEBUG:
    _ = fn_topic_clusters(test_state)


2024-09-27 08:30:49,523 - AInewsbot - INFO - Fetching embeddings for 98 headlines
2024-09-27 08:30:49,973 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-27 08:30:50,298 - AInewsbot - INFO - Sort with nearest_neighbor_sort sort
2024-09-27 08:30:50,331 - AInewsbot - INFO - Perform dimensionality reduction
2024-09-27 08:30:53,578 - AInewsbot - INFO - Cluster with DBSCAN


,id,title_topic_str
0,0,"Sam Altman is setting himself up for a massive $10 billion payday as more OpenAI executives head for the exit (Topics: Big Tech, Chatgpt, Claude, Copilot, Dario Amodei, Economics, Executives, Finance, Funding, Gen AI, Ilya Sutskever, Ipos, Language Models, Open Source, OpenAI, Opinion, Payday, Perplexity, Sam Altman, Stocks)"
1,1,"Source: at an all-hands meeting, Sam Altman said there were no plans for him to get a giant equity stake in OpenAI, calling the information just not true (Topics: Chatgpt, Claude, Dario Amodei, Disinformation, Equity Stake, Finance, Ilya Sutskever, Meeting, Open Source, OpenAI, Sam Altman)"
2,2,"TSMC execs allegedly dismissed OpenAI CEO Sam Altman as 'podcasting bro' (Topics: Big Tech, Chatgpt, Claude, Copilot, Dario Amodei, Dismissal, Ilya Sutskever, Language Models, Open Source, OpenAI, Perplexity, Sam Altman, Semiconductor Chips, Tsmc)"
3,3,"An OpenAI shake-up wouldn't just make Sam Altman richer &mdash; he'd have more control, too (Topics: Big Tech, Chatgpt, Claude, Dario Amodei, Ethics, Executive Control, Gen AI, Ilya Sutskever, Language Models, Mustafa Suleyman, Open Source, OpenAI, Opinion, Perplexity, Richard Socher, Sam Altman, Wealth)"
4,4,"All the top OpenAI figures who have left since the 2023 attempt to oust CEO Sam Altman (Topics: Big Tech, Ceo, Chatgpt, Claude, Dario Amodei, Executive Departures, Gen AI, Ilya Sutskever, Language Models, Mustafa Suleyman, Open Source, OpenAI, Perplexity, Richard Socher, Sam Altman)"


2024-09-27 08:30:54,231 - AInewsbot - INFO - I dub this cluster: OpenAI Leadership and Market Dynamics


,id,title_topic_str
5,5,"ChatGPT's human-like Advanced Voice Mode feature can say Sorry Im late in over 50 languages but the EU is reportedly turning a deaf ear to OpenAI's plea because it can detect user emotion (Topics: Bias And Fairness, Big Tech, Chatbots, Chatgpt, Claude, Cognitive Science, Consciousness, Ethics, Eu, European Union, Gen AI, Governance, Ilya Sutskever, Language, Language Models, Open Source, OpenAI, Perplexity, Policy And Regulation, Politics, Privacy, Privacy & Surveillance, Products, Retrieval Augmented Generation, Richard Socher, Safety And Alignment, Speech Recognition & Synthesis, User Emotion, Virtual Assistants, Voice Mode)"
6,6,"OpenAI Releases ChatGPT Advanced Voice Mode for Paid Subscribers (Topics: Big Tech, Chatbots, Chatgpt, Claude, Code Assistants, Copilot, Dario Amodei, Entertainment, Gen AI, Ilya Sutskever, Language Models, Music, Open Source, OpenAI, Perplexity, Products, Sam Altman, Science, Speech Recognition & Synthesis, Streaming, Subscribers, Virtual Assistants, Voice Mode)"
7,7,"Why I still cant use Advanced Voice Mode even though I pay for ChatGPT Plus (Topics: Chatbots, Chatgpt, Chatgpt Plus, Claude, Code Assistants, Copilot, Gen AI, Issues, Language Models, Open Source, OpenAI, Opinion, Perplexity, Review, Speech Recognition & Synthesis, Virtual Assistants, Voice Mode)"


2024-09-27 08:30:54,687 - AInewsbot - INFO - I dub this cluster: ChatGPT Advanced Voice Mode Features


...

2024-09-27 08:30:56,637 - AInewsbot - INFO - I dub this cluster: Samsung AI Innovations in Consumer Products


,id,title_topic_str
17,17,"Signals Meredith Whittaker: I see AI as born out of surveillance (Topics: AI Doom, Bias And Fairness, Ethics, Facial Recognition, Gen AI, Inequality, Meredith Whittaker, Opinion, Politics, Privacy, Privacy & Surveillance, Safety And Alignment, Science, Surveillance)"
18,18,"AI Thinking Requires Abstractions Clarity to Scale in India (Topics: Abstractions, Cognitive Science, India, Scaling)"
19,19,"How AIs impact is reaching into areas that might surprise you (Topics: Ethics, Impact, Opinion, Safety And Alignment, Society & Culture, Surprising Areas)"


2024-09-27 08:30:57,167 - AInewsbot - INFO - I dub this cluster: AI Ethics and Societal Impact


,id,title_topic_str
20,20,"OpenAI turmoil is a bad look for its huge funding round, VCs say. 'I would vote with my feet.' (Topics: Big Tech, Chatgpt, Claude, Copilot, Dario Amodei, Economics, Finance, Funding, Gen AI, Ilya Sutskever, Ipos, Language Models, Open Source, OpenAI, Opinion, Perplexity, Sam Altman, Stocks, Turmoil, Venture Capital, Venture Capitalists)"
21,21,"In an email to OpenAI's investors, CFO Sarah Friar says the company's current funding round was oversubscribed and is set to close next week (Topics: Big Tech, Cfo, Chatgpt, Claude, Economics, Finance, Funding, Ilya Sutskever, Investors, Ipos, Language Models, OpenAI, Perplexity, Stocks, Venture Capital)"
22,22,"OpenAI's CFO is trying to get the company's investors to keep calm and carry on (Topics: Big Tech, Calm, Cfo, Chatgpt, Claude, Copilot, Dario Amodei, Economics, Finance, Funding, Gen AI, Ilya Sutskever, Investors, Language Models, Open Source, OpenAI, Sam Altman, Stocks, Venture Capital)"


2024-09-27 08:30:57,658 - AInewsbot - INFO - I dub this cluster: OpenAI and Venture Capital Dynamics


In [18]:
# scrape individual pages
def fn_download_pages(state: AgentState) -> AgentState:
    """
    Uses several Selenium browser sessions to download all the pages referenced in the
    state["AIdf"] DataFrame and store their pathnames.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent with the downloaded pages' pathnames stored in the `state["AIdf"]` DataFrame.
    """
    log("Queuing URLs for scraping")
    AIdf = pd.DataFrame(state['AIdf'])
    queue = multiprocessing.Queue()

    count = 0
    for row in AIdf.itertuples():
#         if row.cluster < 999:
        queue.put((row.id, row.url, row.title))
        count +=1
    # scrape urls in queue asynchronously
    num_browsers = 4

    callable = process_url_queue_factory(queue)

    log(f"fetching {count} pages using {num_browsers} browsers")
    saved_pages = launch_drivers(num_browsers, callable)

    pages_df = pd.DataFrame(saved_pages)
    if len(pages_df):
        pages_df.columns = ['id', 'url', 'title', 'path']

        try: # for idempotency
            AIdf = AIdf.drop(columns=['path'])
        except:
            pass        
        AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")
    state["AIdf"] = AIdf.to_dict()
    return state


if DEBUG:
    _ = fn_download_pages(test_state)



2024-09-27 08:30:57,708 - AInewsbot - INFO - Queuing URLs for scraping
2024-09-27 08:30:57,712 - AInewsbot - INFO - fetching 98 pages using 4 browsers
2024-09-27 08:30:57,713 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:30:57,714 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:30:57,715 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:30:57,715 - AInewsbot - INFO - get_driver - 64567 Initializing webdriver
2024-09-27 08:31:14,564 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:31:14,565 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:31:14,566 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:31:14,566 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-09-27 08:31:14,566 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-09-27 08:31:14,567 - AInewsbot - INFO - get_driver - Ini

2024-09-27 08:32:31,737 - AInewsbot - INFO - Processing https://news.google.com/read/CBMimwFBVV95cUxORWFzaVR2ay1MSzhSUjF3RnJNQ3BFSC1kOTJXNEpCV0VhTGYzY1lvQWgzWDNmSDBvdGp5enVnN0kzdDZGSnVUMVVENEpPNmVhRU9kRTk3dlRDWHRNNlgxWmswUG9KVkItaF95WDhTZmlGdVRSWU9qMWhoNHZzMDBkdko0d1ZZV19KUnR1MG43dkIyTjQ5UU0yUDN2QQ
2024-09-27 08:32:31,738 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMimwFBVV95cUxORWFzaVR2ay1MSzhSUjF3RnJNQ3BFSC1kOTJXNEpCV0VhTGYzY1lvQWgzWDNmSDBvdGp5enVnN0kzdDZGSnVUMVVENEpPNmVhRU9kRTk3dlRDWHRNNlgxWmswUG9KVkItaF95WDhTZmlGdVRSWU9qMWhoNHZzMDBkdko0d1ZZV19KUnR1MG43dkIyTjQ5UU0yUDN2QQ) - starting get_url https://news.google.com/read/CBMimwFBVV95cUxORWFzaVR2ay1MSzhSUjF3RnJNQ3BFSC1kOTJXNEpCV0VhTGYzY1lvQWgzWDNmSDBvdGp5enVnN0kzdDZGSnVUMVVENEpPNmVhRU9kRTk3dlRDWHRNNlgxWmswUG9KVkItaF95WDhTZmlGdVRSWU9qMWhoNHZzMDBkdko0d1ZZV19KUnR1MG43dkIyTjQ5UU0yUDN2QQ
2024-09-27 08:32:32,899 - AInewsbot - INFO - get_url(OpenAI Releases ChatGPT Advanced Voice Mode for Paid Subscribers) - Saving OpenAI_Rele

2024-09-27 08:32:56,820 - AInewsbot - INFO - get_url(Samsung trickles latest AI functions to mid-range) - Saving Samsung_trickles_latest_AI_functions_to_mid-range_20240927_083256.html as utf-8
2024-09-27 08:32:56,821 - AInewsbot - INFO - Processing https://analyticsindiamag.com/ai-breakthroughs/ai-thinking-requires-abstractions-clarity-to-scale-in-india/
2024-09-27 08:32:56,821 - AInewsbot - INFO - get_url(https://analyticsindiamag.com/ai-breakthroughs/ai-thinking-requires-abstractions-clarity-to-scale-in-india/) - starting get_url https://analyticsindiamag.com/ai-breakthroughs/ai-thinking-requires-abstractions-clarity-to-scale-in-india/
2024-09-27 08:32:59,361 - AInewsbot - INFO - get_url(Galaxy S24 Series Expands With S24 FE: A Premium Experience That Makes Full Galaxy AI Capabilities Attainable for More Users) - Saving Galaxy_S24_Series_Expands_With_S24_FE__A_Premium_Experience_That_Makes_Full_Galaxy_AI_Capabilities_Attainable_for_More_Users_20240927_083259.html as utf-8
2024-09-27 

2024-09-27 08:33:26,432 - AInewsbot - INFO - get_url(Meta emphasizes AI & AR innovations during developer event as metaverse hype dissipates) - Saving Meta_emphasizes_AI___AR_innovations_during_developer_event_as_metaverse_hype_dissipates_20240927_083326.html as utf-8
2024-09-27 08:33:26,433 - AInewsbot - INFO - Processing https://www.ft.com/content/2243c45a-438e-49ec-8112-7a094a0b3c49
2024-09-27 08:33:26,435 - AInewsbot - INFO - get_url(https://www.ft.com/content/2243c45a-438e-49ec-8112-7a094a0b3c49) - starting get_url https://www.ft.com/content/2243c45a-438e-49ec-8112-7a094a0b3c49
2024-09-27 08:33:26,752 - AInewsbot - INFO - get_url(Nuclear Power Is the New A.I. Trade. What Could Possibly Go Wrong?) - Saving Nuclear_Power_Is_the_New_A_I__Trade__What_Could_Possibly_Go_Wrong__20240927_083326.html as utf-8
2024-09-27 08:33:26,753 - AInewsbot - INFO - Processing https://www.techmonitor.ai/digital-economy/ai-and-automation/accentures-q4-fy24-earnings-exceed-estimates-driven-by-gen-ai-tech

2024-09-27 08:34:01,635 - AInewsbot - INFO - get_url(Why Salesforce Chose London for Its First AI Center and $4 Billion AI Investment) - Saving Why_Salesforce_Chose_London_for_Its_First_AI_Center_and__4_Billion_AI_Investment_20240927_083401.html as utf-8
2024-09-27 08:34:01,637 - AInewsbot - INFO - Processing https://www.forbes.com/sites/lanceeliot/2024/09/27/this-simple-prompting-act-surprisingly-befuddles-advanced-ai-especially-openai-new-o1-model/
2024-09-27 08:34:01,638 - AInewsbot - INFO - get_url(https://www.forbes.com/sites/lanceeliot/2024/09/27/this-simple-prompting-act-surprisingly-befuddles-advanced-ai-especially-openai-new-o1-model/) - starting get_url https://www.forbes.com/sites/lanceeliot/2024/09/27/this-simple-prompting-act-surprisingly-befuddles-advanced-ai-especially-openai-new-o1-model/
2024-09-27 08:34:01,932 - AInewsbot - INFO - get_url(Neo-Nazis are using AI to rebrand Hitler for a new generation) - Saving Neo-Nazis_are_using_AI_to_rebrand_Hitler_for_a_new_generati

2024-09-27 08:34:31,887 - AInewsbot - INFO - get_url(How to Summarize Your Excel Spreadsheets With AI) - Saving How_to_Summarize_Your_Excel_Spreadsheets_With_AI_20240927_083431.html as utf-8
2024-09-27 08:34:31,889 - AInewsbot - INFO - Processing https://www.geeky-gadgets.com/genai-agents-ai-agent-repository/
2024-09-27 08:34:31,889 - AInewsbot - INFO - get_url(https://www.geeky-gadgets.com/genai-agents-ai-agent-repository/) - starting get_url https://www.geeky-gadgets.com/genai-agents-ai-agent-repository/
2024-09-27 08:34:35,308 - AInewsbot - INFO - get_url(How to create your own custom chatbot with Gemini Gems) - Saving How_to_create_your_own_custom_chatbot_with_Gemini_Gems_20240927_083435.html as utf-8
2024-09-27 08:34:35,311 - AInewsbot - INFO - Processing https://www.bbc.co.uk/news/videos/cdje9nz7ll1o
2024-09-27 08:34:35,311 - AInewsbot - INFO - get_url(https://www.bbc.co.uk/news/videos/cdje9nz7ll1o) - starting get_url https://www.bbc.co.uk/news/videos/cdje9nz7ll1o
2024-09-27 08:3

2024-09-27 08:35:01,815 - AInewsbot - INFO - get_url(VentureBeat Special Issue - Tailoring AI Infrastructure) - Saving VentureBeat_Special_Issue_-_Tailoring_AI_Infrastructure_20240927_083501.html as utf-8
2024-09-27 08:35:01,817 - AInewsbot - INFO - Processing https://hackernoon.com/bytedance-bets-big-a-strategic-exit-from-the-social-media-circus-with-ai-video
2024-09-27 08:35:01,817 - AInewsbot - INFO - get_url(https://hackernoon.com/bytedance-bets-big-a-strategic-exit-from-the-social-media-circus-with-ai-video) - starting get_url https://hackernoon.com/bytedance-bets-big-a-strategic-exit-from-the-social-media-circus-with-ai-video
2024-09-27 08:35:09,065 - AInewsbot - INFO - get_url(Can AI show compassion? Teaching AI to care may teach us to be more human ourselves) - Saving Can_AI_show_compassion__Teaching_AI_to_care_may_teach_us_to_be_more_human_ourselves_20240927_083509.html as utf-8
2024-09-27 08:35:09,067 - AInewsbot - INFO - Processing https://t.co/cLCfgoLC8D
2024-09-27 08:35:09

2024-09-27 08:35:42,578 - AInewsbot - INFO - get_url(https://www.theverge.com/2024/9/27/24255722/google-gemini-gmail-contextual-smart-replies-availability) - starting get_url https://www.theverge.com/2024/9/27/24255722/google-gemini-gmail-contextual-smart-replies-availability
2024-09-27 08:35:44,698 - AInewsbot - INFO - get_url(ChatGPT vulnerability allows hackers to record your sessions indefinitely) - Saving ChatGPT_vulnerability_allows_hackers_to_record_your_sessions_indefinitely_20240927_083544.html as utf-8
2024-09-27 08:35:44,704 - AInewsbot - INFO - Processing https://skift.com/2024/09/27/mallorcas-visitor-boom-hotels-and-ai-and-southwests-optimism/
2024-09-27 08:35:44,704 - AInewsbot - INFO - get_url(https://skift.com/2024/09/27/mallorcas-visitor-boom-hotels-and-ai-and-southwests-optimism/) - starting get_url https://skift.com/2024/09/27/mallorcas-visitor-boom-hotels-and-ai-and-southwests-optimism/
2024-09-27 08:35:45,406 - AInewsbot - INFO - get_url(FTC sues five AI outfits  a

2024-09-27 08:36:14,762 - AInewsbot - INFO - Processing https://www.businessinsider.com/one-chipmaker-finally-reaping-the-benefits-of-ai-chip-boom-2024-9
2024-09-27 08:36:14,762 - AInewsbot - INFO - get_url(https://www.businessinsider.com/one-chipmaker-finally-reaping-the-benefits-of-ai-chip-boom-2024-9) - starting get_url https://www.businessinsider.com/one-chipmaker-finally-reaping-the-benefits-of-ai-chip-boom-2024-9
2024-09-27 08:36:18,136 - AInewsbot - INFO - get_url(You can now chat and play games with Duolingos AI characters to learn Spanish or French) - Saving You_can_now_chat_and_play_games_with_Duolingos_AI_characters_to_learn_Spanish_or_French_20240927_083618.html as utf-8
2024-09-27 08:36:18,138 - AInewsbot - INFO - Processing https://www.bloomberg.com/news/articles/2024-09-27/apple-investors-unfazed-by-tepid-demand-for-ai-powered-iphone
2024-09-27 08:36:18,139 - AInewsbot - INFO - get_url(https://www.bloomberg.com/news/articles/2024-09-27/apple-investors-unfazed-by-tepid-de

2024-09-27 08:36:45,530 - AInewsbot - INFO - Processing https://www.forbes.com/councils/forbestechcouncil/2024/09/27/an-enterprises-guide-to-getting-started-with-generative-ai/
2024-09-27 08:36:45,530 - AInewsbot - INFO - get_url(https://www.forbes.com/councils/forbestechcouncil/2024/09/27/an-enterprises-guide-to-getting-started-with-generative-ai/) - starting get_url https://www.forbes.com/councils/forbestechcouncil/2024/09/27/an-enterprises-guide-to-getting-started-with-generative-ai/
2024-09-27 08:36:50,379 - AInewsbot - INFO - get_url(Listen: Google's hidden AI tool turns your text into stunningly lifelike podcasts - for free) - Saving Listen__Google_s_hidden_AI_tool_turns_your_text_into_stunningly_lifelike_podcasts_-_for_free_20240927_083650.html as utf-8
2024-09-27 08:36:50,382 - AInewsbot - INFO - Processing https://www.npr.org/2024/09/27/nx-s1-5126584/linkedin-is-rolling-back-its-use-of-artificial-intelligence
2024-09-27 08:36:50,383 - AInewsbot - INFO - get_url(https://www.npr

In [19]:
# summarize individual pages

def fn_summarize_pages(state: AgentState) -> AgentState:
    """
    Reads all the articles, summarizes each one using a ChatGPT prompt, and sends an email with the summaries.

    Args:
        state (AgentState): The current state of the agent.

    Returns:
        AgentState: The updated state of the agent.

    """
    log("Starting summarize")
    AIdf = pd.DataFrame(state['AIdf'])
    responses = asyncio.run(fetch_all_summaries(AIdf))
    log(f"Received {len(responses)} summaries")
    response_dict = {}
    for i, response in responses:
        try:
            response_str = response["choices"][0]["message"]["content"]
            response_dict[i] = response_str
        except Exception as exc:
            print(exc)

    markdown_str = ''
    bullets = []

    for i, row in enumerate(AIdf.itertuples()):
        topics = []
        if row.cluster_name:
            topics.append(row.cluster_name)
        if row.topic_str:
            topics.append(row.topic_str)
        topic_str = ", ".join(topics)

        mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
        bullets.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
        display(Markdown(mdstr.replace("$","\\$")))
        markdown_str += mdstr

    state['bullets'] = bullets
    # Convert Markdown to HTML
    html_str = markdown.markdown(markdown_str, extensions=['extra'])
    # save bullets
    with open('bullets.md', 'w') as f:
        f.write(markdown_str)
    # send email
    log("Sending bullet points email")
    subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
    send_gmail(subject, html_str)

    return state


if DEBUG:
    _ = fn_summarize_pages(test_state)



2024-09-27 08:37:07,934 - AInewsbot - INFO - Starting summarize
2024-09-27 08:37:07,978 - AInewsbot - INFO - fetch_all_summaries - Page title: Sam Altman is setting himself up for a massive payday | Fortune
Social card title: Sam Altman is setting himself up for a massive $10 billion payday as more OpenAI executives head for the exit
Social card description: The OpenAI CEO stands to become one of the world's richest people.

2024-09-27 08:37:08,094 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI's Sam Altman tells employees he didn't get 'giant equity stake'
Social card title: Sam Altman tells OpenAI staff there's no plan for him to receive a 'giant equity stake' in company
Social card description: At an all-hands meeting Thursday, OpenAI CEO Sam Altman denied reports that he received a "giant equity stake" in the company.

2024-09-27 08:37:08,459 - AInewsbot - INFO - fetch_all_summaries - Page title: TSMC execs allegedly dismissed Sam Altman as ‘podcasting bro’ — OpenAI 

2024-09-27 08:37:09,535 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI's CFO Is Trying to Get Its Investors to Keep Calm and Carry on - Business Insider
Social card title: OpenAI's CFO is trying to get the company's investors to keep calm and carry on
Social card description: Friar wrote in an email that OpenAI's funding round for $6.5 billion is ready to close soon and is already oversubscribed, CNBC reported.

2024-09-27 08:37:09,579 - AInewsbot - INFO - fetch_all_summaries - Page title: Why California Gov. Gavin Newsom should sign an AI model regulation bill - Bulletin of the Atomic Scientists
Social card title: Why California Gov. Gavin Newsom should sign an AI model regulation bill
Social card description: California’s Safe and Secure Innovation for Frontier Artificial Intelligence Models Act, currently with Governor Newsom awaiting his signature, requires that developers of the most powerful AI systems write and follow safety and security protocols to prevent criti

2024-09-27 08:37:10,622 - AInewsbot - INFO - fetch_all_summaries - Page title: Apple Intelligence's AI custom emoji creator is going to be even quicker and easier than we expected | TechRadar
Social card title: Apple Intelligence's AI custom emoji creator is going to be even quicker and easier than we expected
Social card description: Genmoji is coming

2024-09-27 08:37:10,668 - AInewsbot - INFO - fetch_all_summaries - Page title: The Modern Job Hunt: AI Interviewing AI | HackerNoon
Social card title: The Modern Job Hunt: AI Interviewing AI | HackerNoon
Social card description: The rise of AI is fundamentally changing the recruiting process and raising urgent ethical questions.

2024-09-27 08:37:10,705 - AInewsbot - INFO - fetch_all_summaries - Page title: Sundar Pichai-Led Google Rehires Its Top AI Talent For A Whopping $2.7 Billion Paycheck! - Tech
Social card title: Sundar Pichai-Led Google Rehires Its Top AI Talent For A Whopping $2.7 Billion Paycheck!
Social card description: The 

2024-09-27 08:37:11,469 - AInewsbot - INFO - fetch_all_summaries - Page title: Data harvesting superapp says LLM tamed its unruly data • The Register
Social card title: Data harvesting superapp says LLM tamed its unruly data
Social card description: Engineers at Grab don't need to ask each other questions any more

2024-09-27 08:37:11,506 - AInewsbot - INFO - fetch_all_summaries - Page title: China’s Plan to Make AI Watermarks Happen | WIRED
Social card title: China’s Plan to Make AI Watermarks Happen
Social card description: New Chinese regulation attempts to define how AI content labeling should work and stamp out AI-generated disinformation.

2024-09-27 08:37:11,551 - AInewsbot - INFO - fetch_all_summaries - Page title: ChatGPT vulnerability allows hackers to record your sessions indefinitely | TechSpot
Social card title: ChatGPT vulnerability allows hackers to record your sessions indefinitely
Social card description: Security research Johann Rehberger recently discovered a way to 

2024-09-27 08:37:12,431 - AInewsbot - INFO - fetch_all_summaries - Page title: Apple rejects new AI pact in EU, despite support from OpenAI, Google, more - 9to5Mac
Social card title: Apple rejects new AI pact in EU, despite support from OpenAI, Google, more - 9to5Mac
Social card description: Apple has refused to sign a new AI pact in the EU, despite it receiving support from several other major AI players like Google and OpenAI.

2024-09-27 08:37:12,452 - AInewsbot - INFO - fetch_all_summaries - Page title: Mitigating AI Risks in Pharma Needs a New Governance Framework
Social card title: Mitigating AI Risks in Pharma Needs a New Governance Framework
Social card description: Winston &amp; Strawn’s Keerthika Subramanian and Mollie Goldfarb share tips for pharma leaders to internally manage legal risks from AI use.

2024-09-27 08:37:12,485 - AInewsbot - INFO - fetch_all_summaries - Page title: Bill Gates Says AI Positives Outweigh Concerns: 'We'll Be Experiencing Increased Productivity In

...

[95. An Enterprises Guide To Getting Started With Generative AI - Forbes](https://www.forbes.com/councils/forbestechcouncil/2024/09/27/an-enterprises-guide-to-getting-started-with-generative-ai/)  

 Enterprise, Gen AI, Generative AI, Guide, Implementation, Language Models, Retrieval Augmented Generation  

- Enterprises must address four critical factors before leveraging Generative AI (GenAI): knowledge gaps among employees, infrastructure challenges, and data management issues.
- To overcome the knowledge problem, companies should recruit skilled data scientists, provide training for existing staff, and foster cross-functional collaboration.
- Robust data governance and centralized data repositories are essential for managing data quality, security, and compliance, ensuring effective use of GenAI applications. 



[96. LinkedIn is rolling back its use ofartificial intelligence - NPR](https://www.npr.org/2024/09/27/nx-s1-5126584/linkedin-is-rolling-back-its-use-of-artificial-intelligence)  

 Big Tech, Gen AI, Linkedin, Rollback, Science  

- LinkedIn is scaling back its implementation of artificial intelligence in its offerings.
- The discussion involves insights from Christopher Zara of Fast Company, who has been monitoring LinkedIn's AI developments.
- The audio segment features NPR's Leila Fadel interviewing Zara about these changes. 



[97. Meta explores neural control and AI beats bot detectors - ZDNet](https://www.zdnet.com/article/meta-harnesses-mind-control-and-ai-beats-bot-detectors/#ftag=RSSbaffb68)  

 Big Tech, Bot Detection, Brain-Computer Interfaces, Cognitive Science, Gen AI, Meta, Neural Control, Privacy & Surveillance, Safety And Alignment, Science  

- Meta has launched Orion AR glasses, which utilize holograms for notifications and an innovative neural interface for gesture commands, enhancing user interaction and social awareness.
- Swiss researchers have developed an AI that can pass reCAPTCHA tests with 100% accuracy, potentially rendering current security measures obsolete and prompting the need for more advanced verification methods.
- Meta also upgraded Ray-Bans to include features like remembering viewed locations and live translation, signaling a trend towards more integrated AI in everyday wearables. 



[98. Why Nvidia Stock Could Lag Behind A Key AI Energy Company - Google News](https://news.google.com/read/CBMirgFBVV95cUxOOHlYekFNTkRQMUVZaDhyMHUxNzlKcWhDTjdGTENTak5BWFI3VTNabXo2aE41QUpxRzR1eTk2cVdCSE5taHJfSGtyRjZkaTY0UThCb3FXYktMSFVSSGlJam9SS0FDVU1zcjBXRXhtd1A1eHRWUDAtUWZtUU1acjhlNEc1bHJDVkdCV1NHOFBNWExOSE1pSDN4YlN0UWJHdzFZRmFDY1ZXS0VSa2dIRlE)  

 Climate, Economics, Energy, Energy Company, Finance, Gen AI, Hardware, Nvidia, Opinion, Semiconductor Chips, Stock, Stocks  

- Constellation Energy's stock has surged due to a significant deal with Microsoft for nuclear power, whereas Nvidia's revenue growth is slowing, prompting analysts to consider Constellation as a potentially better investment.
- The demand for energy from data centers is expected to grow significantly, which could favor Constellation's business model, while Nvidia's AI chip revenue growth is predicted to decelerate.
- Analysts have raised price targets for Constellation significantly post-Microsoft deal, indicating optimism for its revenue growth potential and demand for clean energy. 



2024-09-27 08:37:17,033 - AInewsbot - INFO - Sending bullet points email


In [20]:
def fn_propose_cats(state: AgentState) -> AgentState:
    # ask chatgpt for top categories

    model = ChatOpenAI(
        model=MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template("{p}") | model | SimpleJsonOutputParser()
    response = chain.invoke({ "p": TOP_CATEGORIES_PROMPT + "\n\n".join(state["bullets"])})
    suggested_categories = []
    for k, v in response.items():
        suggested_categories.extend(v)
    state["cluster_topics"] = list(set(state["cluster_topics"] + suggested_categories))
    state["cluster_topics"].sort()
    return state


if DEBUG:
    _ = fn_propose_cats(test_state)
    print("edit the following proposed topic list and update state['cluster_topics']:")
    print('"' + '",\n"'.join(test_state["cluster_topics"]) + '"')


2024-09-27 08:37:29,621 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


edit the following proposed topic list and update state['cluster_topics']:
"AI Ethics and Societal Impact",
"AI Models and Big Tech Innovations",
"AI chip innovation challenges",
"AI in India's scaling challenges",
"AI-generated video deepfakes",
"Accenture Q4 earnings from Gen AI",
"California AI regulation bill",
"ChatGPT Advanced Voice Mode",
"ChatGPT Advanced Voice Mode Features",
"ChatGPT and AI Conversations",
"Figure's GPT mortgage tool",
"Google rehiring AI talent",
"Hugging Face 1M models milestone",
"Meredith Whittaker on AI surveillance",
"Meta's AI and AR innovations",
"Meta's Llama 3.2 Vision AI",
"Nuclear power and AI energy demands",
"OpenAI Leadership and Market Dynamics",
"OpenAI and Venture Capital Dynamics",
"OpenAI executive turnover",
"OpenAI funding round turmoil",
"OpenAI restructuring",
"Sam Altman's $10B payday",
"Samsung AI Innovations in Consumer Products",
"Samsung Galaxy S24 AI features",
"TSMC dismisses Altman",
"Zerodha CTO on AGI timeline"


------------
------------


In [24]:
test_state["cluster_topics"] = ["AI Ethics",
"New AI Models",
"AI chip innovation ",
"India",
"Deepfakes",
"Accenture",
"California AI regulation bill",
"ChatGPT Advanced Voice Mode",
"Figure's GPT mortgage tool",
"Google rehiring AI talent",
"Hugging Face",
"Meredith Whittaker on AI surveillance",
"Meta's AI and AR innovations",
"Nuclear power",
"OpenAI",
"Sam Altman's $10B payday",
"Samsung",
"TSMC",
"AGI"

]


In [25]:
FINAL_SUMMARY_PROMPT = """You are ASA, an advanced summarization assistant designed to
write a compelling summary of news input. You are able to categorize information,
and identify important themes from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in markdown format,
structured according to an input format template.

News items are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - TechCrunch](https://techcrunch.com/2024/09/19/apple-intelligence-is-now-live-in-public-beta-heres-what-it-offers-and-how-to-enable-it)

Topics: Apple, Big Tech, Features, Gen AI, Intelligence, Machine Learning, Products, Public Beta, Virtual Assistants

- Apple Intelligence is now live in public beta for users in the U.S. enrolled in the public beta program, featuring generative AI capabilities like advanced writing tools and a revamped Siri.
- The platform is currently only available in U.S. English and is not accessible in the EU or China due to regulatory issues; it supports iPhone 15 Pro, Pro Max, and the new iPhone 16 line.
- Key features include photo editing tools like "Clean Up," a Smart Reply function in Mail, and improvements to Siri’s understanding and on-device task knowledge.

ASA Output Format Template:

# Engaging-topic-title-1

- item-title-1a - [source-name-1a](item-url-1a)
- item-title-1b - [source-name-1b](item-url-1b)
- item-title-1c - [source-name-1c](item-url-1c)

# Engaging-topic-title-2

- item-title-2a - [source-name-2a](item-url-2a)
- item-title-2b - [source-name-2b](item-url-2b)

Example ASA Output Format:

# A military AI revolution

- Eric Schmidt on AI warfare - [FT](https://www.ft.com/content/fe136479-9504-4588-869f-900f2b3452c4)
- Killer robots are real in Ukraine war. - [Yahoo News](https://uk.news.yahoo.com/ai-killer-robots-warning-ukraine-war-133415411.html)

ASA Instructions:
Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Group news items into related topics.
Each topic should have a snappy, punchy, clever, alliterative, possibly punny title.
Each output item bullet should contain one sentence with one link.
Each topic chould contain the most significant facts from the news items without commentary or elaboration.
Each output item bullet should not repeat points or information from previous bullet points.
You will write each item in the professional but engaging, narrative style of a tech reporter
for a national publication, providing balanced, professional, informative, providing accurate,
clear, concise summaries in a neutral tone.

Check carefully that you only use information provided in the input below, that you include
a link in each output item, and that any bullet point does not repeat information or links previously provided.

Topic suggestions:
{cat_str}

Input:
{bullet_str}

"""

In [26]:
def fn_compose_summary(state: AgentState) -> AgentState:

    cat_str = "\n".join(state['cluster_topics'])
    bullet_str = "\n~~~\n".join(state["bullets"])

    model = ChatOpenAI(
        model=HIGHCOST_MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template(FINAL_SUMMARY_PROMPT) | model | SimpleJsonOutputParser()
    response = chain.invoke({ "cat_str": cat_str, "bullet_str": bullet_str})
    print(response)
    state["summary"] = response.content
    return state


if DEBUG:
    _ = fn_compose_summary(test_state)


2024-09-27 08:48:22,989 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported value: 'temperature' does not support 0.3 with this model. Only the default (1) value is supported.", 'type': 'invalid_request_error', 'param': 'temperature', 'code': 'unsupported_value'}}

In [27]:
#post manually to o1-preview and grab response
cat_str = "\n".join(test_state['cluster_topics'])
bullet_str = "\n~~~\n".join(test_state["bullets"])
final_summary_prompt = FINAL_SUMMARY_PROMPT.format(cat_str=cat_str, bullet_str=bullet_str)
print(final_summary_prompt)


You are ASA, an advanced summarization assistant designed to
write a compelling summary of news input. You are able to categorize information,
and identify important themes from large volumes of news.

ASA Objective:

I will provide today's news items about AI and summary bullet points in markdown format,
structured according to an input format template.

News items are delimited by ~~~

You are tasked with using the news items to create a concise summary of today's most important topics and developments.

You will write an engaging summary of today's news encompassing the most important and frequently
mentioned topics and themes, in an output format provided below.

ASA Input Item Format Template:

[Story-Title-s1 - source-name-s1](url-s1)

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

Example ASA Input Item Format:

[Apple Intelligence is now live in public beta. Heres what it offers and how to enable it. - TechCrunch](https://t

In [28]:
count_tokens(final_summary_prompt)


22054

In [29]:
test_state["summary"] = """
# OpenAI's Altman Adventure: From Nonprofit to Profits

- Sam Altman is poised for a potential $10 billion payday as OpenAI shifts to a for-profit model, raising $6.5 billion at a $150 billion valuation. - [Google News](https://news.google.com/read/CBMib0FVX3lxTE1fWEtwaF8zSDlFamFDR3o4S2l6RDhtV1kyc0JaemlYSHZaYUFObl9xZ3ZrandKOFFrYWRudDJFRHA3QzJfXzhFQ0hmQk01RnRpa1V3NE4tRkY2NGMxb0gwalBPWm5HMWRKVFJtZGdWNA)
- Altman denies plans for a giant equity stake amid executive departures at OpenAI, stating it's "just not true" during an all-hands meeting. - [CNBC](https://www.cnbc.com/2024/09/26/openais-sam-altman-tells-employees-he-didnt-get-giant-equity-stake.html)
- OpenAI's restructuring may increase Altman's control and wealth as it becomes a for-profit benefit corporation. - [Business Insider](https://www.businessinsider.com/sam-altman-openai-restructure-richer-control-2024-9)
- Leadership turnover continues at OpenAI since the 2023 attempt to oust Altman, with multiple high-profile departures. - [Business Insider](https://www.businessinsider.com/openai-leaders-who-left-since-2023-sam-altman-leadership-struggle-2024-9)
- VCs express concern over OpenAI's turmoil impacting its huge funding round, though some remain confident in Altman's leadership. - [Business Insider](https://www.businessinsider.com/openai-implosion-investors-vcs-2024-9)
- OpenAI's CFO reassures investors that the oversubscribed $6.5 billion funding round is set to close next week. - [CNBC](https://www.cnbc.com/2024/09/26/openais-cfo-says-funding-round-should-close-by-next-week-in-letter.html)

# ChatGPT Chatter Blocked by EU's Emotion Notion

- The EU bars OpenAI from launching Advanced Voice mode due to regulations against AI systems that infer emotions. - [Google News](https://news.google.com/read/CBMimAFBVV95cUxQV1BmZFhxMDNPWU1PUHY1QnZiLWlEd0gzeWF4aU9DR3BKc3gxWjhfRy1GWlRLdVpVME9hc1o4RWFNTldCV2dUZGRVcFZtOE5OME9wai11U0pYdFVUbWRtVHVvXzRhSHZUQ2RqMzNqRzFGSE9yYTZ3akQxRkl2WnU0SFA3OWRBbHA5blFCZTRNMmlvaFA4cmtpaw)
- OpenAI releases Advanced Voice mode for paid subscribers in the US, offering conversational interactions with new voices. - [ExtremeTech](https://www.extremetech.com/internet/openai-releases-chatgpt-advanced-voice-mode-for-paid-subscribers)
- EU users are frustrated as they can't access Advanced Voice mode despite paying for ChatGPT Plus, awaiting regulatory compliance. - [BGR](https://bgr.com/tech/im-a-chatgpt-plus-user-but-no-advanced-voice-mode/)

# Meta's Magical Makeover: From Metaverse to Multimodal Marvels

- Meta shifts focus to AI and AR innovations, unveiling the Llama 3.2 AI model and AI-generated avatars at a developer event. - [The Drum](https://www.thedrum.com/news/2024/09/25/meta-emphasizes-ai-and-ar-during-developer-event-signaling-shift-business-priorities)
- Together AI offers free access to Meta’s Llama 3.2 Vision AI model via Hugging Face, enabling image interaction. - [VentureBeat](https://venturebeat.com/ai/heres-how-to-try-metas-new-llama-3-2-with-vision-for-free/)
- Mark Zuckerberg announces Meta AI nearing 500 million users, on track to be the most utilized AI assistant. - [Google News](https://news.google.com/read/CBMilwFBVV95cUxNX3VoN0I5WG04WWgxV21yb1dPQkN6OHVJd2FONGYyZk1DV3l0UEEtaHpncTJMV1h3NzZjOEQ1LWp3cFZzQUFrci1XcVJ3UFVfVXFVdUVhWncweTBsTEtDS2pnTF9Ca3lKX2p1RTFSYXgwRGM2b1JPRlZjcnNMQ2dUa1I1eHZ6aWlkS1h6V0JYamY2QlZhT3JV)
- Meta introduces AI-generated posts tailored to user interests on Facebook and Instagram, opening a Pandora's box. - [Axios](https://www.axios.com/2024/09/27/meta-ai-posts-facebook-instagram)

# Fintech Figures Out: GPT Revolutionizes Mortgages

- Figure launches an AI tool powered by GPT-4, reducing the mortgage lending process from 45 days to just five. - [TechCrunch](https://techcrunch.com/2024/09/26/former-brex-coo-who-now-heads-unicorn-fintech-figure-says-gpt-is-already-upending-the-mortgage-industry/)

# Hugging Face Hits a Million: Model Mania

- Hugging Face surpasses 1 million AI models, driven by community customization and the demand for specialized models. - [Ars Technica](https://arstechnica.com/information-technology/2024/09/ai-hosting-platform-surpasses-1-million-models-for-the-first-time/)

# Samsung Sprinkles AI on Smartphones and Slates

- Samsung unveils the Galaxy S24 FE, bringing flagship AI capabilities to a mid-range smartphone with enhanced cameras. - [Mobile World Live](https://www.mobileworldlive.com/samsung/samsung-trickles-latest-ai-functions-to-mid-range/)
- New Samsung Galaxy S10 tablets include AI features and S Pen support, focusing on AI-driven productivity. - [Google News](https://news.google.com/read/CBMimAFBVV95cUxPUW5DbW5ycldCak5uT0VlUEluYTQ0UDRINDZQdWNDVXZpM3E3a3NWTTF1TXA2Y0xoVXZjWi1xcExFMVpwLWJhcTVtZDdvMnIxZUw5eUl1b1BQd1poLXZHc1J4aEctTEk4QjhtOWpRV2oyWDdsRFktYzc1bFBwVl8wdC0tY2xPS2tNVmVHZ19GQVZuZGoyUzdzcA)

# AI Ethics and Edicts: Regulations Rising

- California's SB 1047 aims to regulate powerful AI systems by requiring safety protocols, awaiting Governor Newsom's signature. - [The Bulletin](https://thebulletin.org/2024/09/why-california-gov-gavin-newsom-should-sign-an-ai-model-regulation-bill/)
- The FTC's "Operation AI Comply" targets companies for misleading AI claims, stretching boundaries on AI oversight. - [Bloomberg Law](https://news.bloomberglaw.com/privacy-and-data-security/ftc-enforcement-sweep-stretches-boundaries-on-ai-oversight)
- A whistleblower warns that OpenAI's shift to for-profit may lead to cutting corners on safety and governance issues. - [Google News](https://news.google.com/read/CBMiywFBVV95cUxPR21HOEU1ZmVxMDB2VnNFM2dBVmM0ZHVWYUZ6cU10QUo3RUJyV3k4ejNXcjRLYVYzZUx6c0huZlZTblBxVEpVbG9lb29XdVdZbTFIeUxGZExYSWpkT0FPMlUyS285bVZrWmFNYmhxekVNTnVIY3JFSnlrdlR0R1N3aG5QUVJfMnRrbVJCSWV0dEJQRi0xWnlvNDlQSzczZl9GTTY5VDBNcHBlUkdQb21maW95V3JFTEt2bnVBcWdTYjlaX0M4YzBQUkxHd9IBywFBVV95cUxQSWxiMzNSTXJOYVBZTEk2YzQzWW4tcnlEYkVtbmZRUVRwM3NLUzBFWHo1cTlveFlsY3UtYjBLVE1INTBwcjdXd2h5TXlSU3RwblhMUzZlcU9BZk9kM2RvREYzdU5ScUQtUnpUejhkN1h5dWR0SzZqRVZhT0hJYmNhY2c3LVJwTUdzN3pmWnh4aWVBYWFjbnB1UkY2UTlLNDl5UkxmS2U5ZzZ3VmF4dW9HSDZ3QUo2ZjR0SXpLWG5zckxzR1dBUEpXWjZ4RQ)

# The AI Talent Tussle: Star Quarterbacks of Tech

- Google rehires AI expert Noam Shazeer for a reported $2.7 billion, highlighting the fierce competition for AI talent. - [Business Insider](https://www.businessinsider.com/google-noam-shazeer-deal-ai-talent-hiring-star-quarterbacks-i2024-9)
- AI geniuses like Shazeer are now seen as the "star quarterbacks" of tech due to their unique skills and contributions. - [Mashable](https://in.mashable.com/tech/82944/sundar-pichai-led-google-rehires-its-top-ai-talent-for-a-whopping-27-billion-paycheck)

# AI's Energy Appetite: Nuclear is the New Neural

- Demand from AI data centers drives a surge in utility stocks, with nuclear power becoming the new AI trade. - [Google News](https://news.google.com/read/CBMifEFVX3lxTE1MNWI3SGxwY0R3ZVJOczdzei1ZZmZ3QWVqaERkTWNpcmFubWdGQlFSVnAzOGlpYWFNZkxhVzhlS2REUVNxM0dGX01ZTVlaVkpycXVmZmpJSTFqbXB1eWhfLUI2Q3pWSHNLMjN1eE9kRVBtcHotSGhoc1Q1OWs)
- Constellation Energy's stock rises after supplying nuclear power to Microsoft's AI data centers. - [Business Insider](https://www.businessinsider.com/one-chipmaker-finally-reaping-the-benefits-of-ai-chip-boom-2024-9)

# Deepfake Dangers: Digital Deceptions on the Rise

- Neo-Nazis use AI to rebrand Hitler, creating AI-edited videos to spread misinformation online. - [The Washington Post](https://www.washingtonpost.com/technology/2024/09/27/neonazis-ai-hitler-videos/)
- AI-generated videos are becoming more realistic, making it crucial to learn how to spot fakes as they improve. - [CNET](https://www.cnet.com/tech/services-and-software/this-ai-generated-video-was-wild-how-to-spot-fakes-as-they-keep-getting-better/#ftag=CAD590a51e)

"""


In [30]:
REWRITE_PROMPT = """You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging AI trends, and the ability to
produce, edit, and curate high-quality content that engages and informs readers. You are
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness,
and coherence, and ensuring its accuracy and relevance.

Objective: The markdown newsletter provided below contains several sections consisting of bullet points.
Carefully review each section of the newsletter. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'OK' if no changes
are needed.

Instructions:
For each section, remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible, sticking to facts without editorial comment.
For each section, review the title. The title should be snappy, punchy, clever, possibly alliterative or punny.
The section title most be short, engaging, clever, and as consistent with the bullets in the section as possible
At the top of the newsletter add an overall title synthesizing several top news items.
Respond with the updated newsletter only in markdown format, or the word 'OK' if no changes are needed.

Newsletter to edit:
{summary}

"""

In [31]:
def fn_rewrite_summary(state: AgentState) -> AgentState:

    model = ChatOpenAI(
        model=HIGHCOST_MODEL,
        temperature=0.3,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    chain = ChatPromptTemplate.from_template(REWRITE_PROMPT) | model | SimpleJsonOutputParser()
    response = chain.invoke({ "summary": state["summary"]})
    print(response)
    state["summary"] = response.content
    return state


if DEBUG:
    _ = fn_rewrite_summary(test_state)


2024-09-27 08:54:16,612 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "Unsupported value: 'temperature' does not support 0.3 with this model. Only the default (1) value is supported.", 'type': 'invalid_request_error', 'param': 'temperature', 'code': 'unsupported_value'}}

In [32]:
rewrite_prompt = (REWRITE_PROMPT.format(summary=test_state["summary"]))
print(rewrite_prompt)


You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging AI trends, and the ability to
produce, edit, and curate high-quality content that engages and informs readers. You are
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness,
and coherence, and ensuring its accuracy and relevance.

Objective: The markdown newsletter provided below contains several sections consisting of bullet points.
Carefully review each section of the newsletter. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'OK' if no changes
are needed.

Instructions:
For each section, remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible, sticking to facts without editorial comment.
For each section, review the title. The title should be snappy, punchy, clever, po

In [33]:
test_state["summmary"] = """
# AI's Big Moves: OpenAI's Profit Pivot, Meta's AI Metamorphosis, and Rising Regulations

## Altman's AI Ascension: OpenAI's Profit Pivot Amid Turmoil

- OpenAI is raising $6.5 billion at a $150 billion valuation, potentially giving Sam Altman a significant payday as the company shifts to a for-profit model. - [Google News](https://news.google.com/read/CBMib0FVX3lxTE1fWEtwaF8zSDlFamFDR3o4S2l6RDhtV1kyc0JaemlYSHZaYUFObl9xZ3ZrandKOFFrYWRudDJFRHA3QzJfXzhFQ0hmQk01RnRpa1V3NE4tRkY2NGMxb0gwalBPWm5HMWRKVFJtZGdWNA), [Business Insider](https://www.businessinsider.com/sam-altman-openai-restructure-richer-control-2024-9)
- Altman denies plans for a giant equity stake amid executive departures, stating it's "just not true" during an all-hands meeting. - [CNBC](https://www.cnbc.com/2024/09/26/openais-sam-altman-tells-employees-he-didnt-get-giant-equity-stake.html)
- Multiple high-profile leaders have departed OpenAI since 2023, continuing leadership turnover. - [Business Insider](https://www.businessinsider.com/openai-leaders-who-left-since-2023-sam-altman-leadership-struggle-2024-9)
- Despite VCs' concerns over turmoil impacting the funding round, OpenAI's CFO says the oversubscribed $6.5 billion funding is set to close next week. - [Business Insider](https://www.businessinsider.com/openai-implosion-investors-vcs-2024-9), [CNBC](https://www.cnbc.com/2024/09/26/openais-cfo-says-funding-round-should-close-by-next-week-in-letter.html)

## ChatGPT Chatter Blocked by EU's Emotion Notion

- The EU prohibits OpenAI from launching Advanced Voice mode due to regulations against AI that infer emotions. - [Google News](https://news.google.com/read/CBMimAFBVV95cUxQV1BmZFhxMDNPWU1PUHY1QnZiLWlEd0gzeWF4aU9DR3BKc3gxWjhfRy1GWlRLdVpVME9hc1o4RWFNTldCV2dUZGRVcFZtOE5OME9wai11U0pYdFVUbWRtVHVvXzRhSHZUQ2RqMzNqRzFGSE9yYTZ3akQxRkl2WnU0SFA3OWRBbHA5blFCZTRNMmlvaFA4cmtpaw)
- OpenAI releases Advanced Voice mode for paid U.S. subscribers, offering conversational interactions with new voices. - [ExtremeTech](https://www.extremetech.com/internet/openai-releases-chatgpt-advanced-voice-mode-for-paid-subscribers)
- EU ChatGPT Plus users can't access Advanced Voice mode due to regulatory issues, causing frustration. - [BGR](https://bgr.com/tech/im-a-chatgpt-plus-user-but-no-advanced-voice-mode/)

## Meta's Magical Makeover: From Metaverse to Multimodal Marvels

- Meta shifts focus to AI and AR innovations, unveiling the Llama 3.2 AI model and AI-generated avatars at a developer event. - [The Drum](https://www.thedrum.com/news/2024/09/25/meta-emphasizes-ai-and-ar-during-developer-event-signaling-shift-business-priorities)
- Llama 3.2 Vision AI model is now accessible via Hugging Face, enabling image interaction. - [VentureBeat](https://venturebeat.com/ai/heres-how-to-try-metas-new-llama-3-2-with-vision-for-free/)
- Mark Zuckerberg announces Meta AI nearing 500 million users, set to be the most utilized AI assistant. - [Google News](https://news.google.com/read/CBMilwFBVV95cUxNX3VoN0I5WG04WWgxV21yb1dPQkN6OHVJd2FONGYyZk1DV3l0UEEtaHpncTJMV1h3NzZjOEQ1LWp3cFZzQUFrci1XcVJ3UFVfVXFVdUVhWncweTBsTEtDS2pnTF9Ca3lKX2p1RTFSYXgwRGM2b1JPRlZjcnNMQ2dUa1I1eHZ6aWlkS1h6V0JYamY2QlZhT3JV)
- Meta introduces AI-generated posts tailored to user interests on Facebook and Instagram. - [Axios](https://www.axios.com/2024/09/27/meta-ai-posts-facebook-instagram)

## Fintech Figures Out: GPT Revolutionizes Mortgages

- Figure launches an AI tool powered by GPT-4, reducing the mortgage lending process from 45 days to just five. - [TechCrunch](https://techcrunch.com/2024/09/26/former-brex-coo-who-now-heads-unicorn-fintech-figure-says-gpt-is-already-upending-the-mortgage-industry/)

## Hugging Face Hits a Million: Model Mania

- Hugging Face surpasses 1 million AI models, driven by community customization and the demand for specialized models. - [Ars Technica](https://arstechnica.com/information-technology/2024/09/ai-hosting-platform-surpasses-1-million-models-for-the-first-time/)

## Samsung Sprinkles AI on Smartphones and Slates

- Samsung unveils the Galaxy S24 FE smartphone and Galaxy S10 tablets, bringing flagship AI capabilities and AI-driven productivity features to mid-range devices. - [Mobile World Live](https://www.mobileworldlive.com/samsung/samsung-trickles-latest-ai-functions-to-mid-range/), [Google News](https://news.google.com/read/CBMimAFBVV95cUxPUW5DbW5ycldCak5uT0VlUEluYTQ0UDRINDZQdWNDVXZpM3E3a3NWTTF1TXA2Y0xoVXZjWi1xcExFMVpwLWJhcTVtZDdvMnIxZUw5eUl1b1BQd1poLXZHc1J4aEctTEk4QjhtOWpRV2oyWDdsRFktYzc1bFBwVl8wdC0tY2xPS2tNVmVHZ19GQVZuZGoyUzdzcA)

## AI Ethics and Edicts: Regulations Rising

- California's SB 1047 seeks to regulate powerful AI systems by requiring safety protocols; awaiting Governor Newsom's signature. - [The Bulletin](https://thebulletin.org/2024/09/why-california-gov-gavin-newsom-should-sign-an-ai-model-regulation-bill/)
- FTC's "Operation AI Comply" targets companies for misleading AI claims, expanding AI oversight. - [Bloomberg Law](https://news.bloomberglaw.com/privacy-and-data-security/ftc-enforcement-sweep-stretches-boundaries-on-ai-oversight)
- Whistleblower warns OpenAI's shift to for-profit may compromise safety and governance. - [Google News](https://news.google.com/read/CBMiywFBVV95cUxPR21HOEU1ZmVxMDB2VnNFM2dBVmM0ZHVWYUZ6cU10QUo3RUJyV3k4ejNXcjRLYVYzZUx6c0huZlZTblBxVEpVbG9lb29XdVdZbTFIeUxGZExYSWpkT0FPMlUyS285bVZrWmFNYmhxekVNTnVIY3JFSnlrdlR0R1N3aG5QUVJfMnRrbVJCSWV0dEJQRi0xWnlvNDlQSzczZl9GTTY5VDBNcHBlUkdQb21maW95V3JFTEt2bnVBcWdTYjlaX0M4YzBQUkxHd9IBywFBVV95cUxQSWxiMzNSTXJOYVBZTEk2YzQzWW4tcnlEYkVtbmZRUVRwM3NLUzBFWHo1cTlveFlsY3UtYjBLVE1INTBwcjdXd2h5TXlSU3RwblhMUzZlcU9BZk9kM2RvREYzdU5ScUQtUnpUejhkN1h5dWR0SzZqRVZhT0hJYmNhY2c3LVJwTUdzN3pmWnh4aWVBYWFjbnB1UkY2UTlLNDl5UkxmS2U5ZzZ3VmF4dW9HSDZ3QUo2ZjR0SXpLWG5zckxzR1dBUEpXWjZ4RQ)

## The AI Talent Tussle: Star Quarterbacks of Tech

- Google rehires AI expert Noam Shazeer for a reported $2.7 billion, highlighting fierce competition for AI talent. - [Business Insider](https://www.businessinsider.com/google-noam-shazeer-deal-ai-talent-hiring-star-quarterbacks-i2024-9)
- AI experts like Shazeer are considered the "star quarterbacks" of tech due to their unique skills. - [Mashable](https://in.mashable.com/tech/82944/sundar-pichai-led-google-rehires-its-top-ai-talent-for-a-whopping-27-billion-paycheck)

## AI's Energy Appetite: Nuclear is the New Neural

- AI data centers' energy demands drive surge in utility stocks, making nuclear power a new AI trade. - [Google News](https://news.google.com/read/CBMifEFVX3lxTE1MNWI3SGxwY0R3ZVJOczdzei1ZZmZ3QWVqaERkTWNpcmFubWdGQlFSVnAzOGlpYWFNZkxhVzhlS2REUVNxM0dGX01ZTVlaVkpycXVmZmpJSTFqbXB1eWhfLUI2Q3pWSHNLMjN1eE9kRVBtcHotSGhoc1Q1OWs)
- Constellation Energy's stock rises after supplying nuclear power to Microsoft's AI data centers. - [Business Insider](https://www.businessinsider.com/one-chipmaker-finally-reaping-the-benefits-of-ai-chip-boom-2024-9)

## Deepfake Dangers: Digital Deceptions on the Rise

- Neo-Nazis use AI to rebrand Hitler, creating AI-edited videos to spread misinformation online. - [The Washington Post](https://www.washingtonpost.com/technology/2024/09/27/neonazis-ai-hitler-videos/)
- AI-generated videos are becoming more realistic, making it crucial to learn how to spot fakes as they improve. - [CNET](https://www.cnet.com/tech/services-and-software/this-ai-generated-video-was-wild-how-to-spot-fakes-as-they-keep-getting-better/#ftag=CAD590a51e)
"""

In [38]:
display(Markdown(test_state["summmary"].replace("$","\\\\$")))



# AI's Big Moves: OpenAI's Profit Pivot, Meta's AI Metamorphosis, and Rising Regulations

## Altman's AI Ascension: OpenAI's Profit Pivot Amid Turmoil

- OpenAI is raising \\$6.5 billion at a \\$150 billion valuation, potentially giving Sam Altman a significant payday as the company shifts to a for-profit model. - [Google News](https://news.google.com/read/CBMib0FVX3lxTE1fWEtwaF8zSDlFamFDR3o4S2l6RDhtV1kyc0JaemlYSHZaYUFObl9xZ3ZrandKOFFrYWRudDJFRHA3QzJfXzhFQ0hmQk01RnRpa1V3NE4tRkY2NGMxb0gwalBPWm5HMWRKVFJtZGdWNA), [Business Insider](https://www.businessinsider.com/sam-altman-openai-restructure-richer-control-2024-9)
- Altman denies plans for a giant equity stake amid executive departures, stating it's "just not true" during an all-hands meeting. - [CNBC](https://www.cnbc.com/2024/09/26/openais-sam-altman-tells-employees-he-didnt-get-giant-equity-stake.html)
- Multiple high-profile leaders have departed OpenAI since 2023, continuing leadership turnover. - [Business Insider](https://www.businessinsider.com/openai-leaders-who-left-since-2023-sam-altman-leadership-struggle-2024-9)
- Despite VCs' concerns over turmoil impacting the funding round, OpenAI's CFO says the oversubscribed \\$6.5 billion funding is set to close next week. - [Business Insider](https://www.businessinsider.com/openai-implosion-investors-vcs-2024-9), [CNBC](https://www.cnbc.com/2024/09/26/openais-cfo-says-funding-round-should-close-by-next-week-in-letter.html)

## ChatGPT Chatter Blocked by EU's Emotion Notion

- The EU prohibits OpenAI from launching Advanced Voice mode due to regulations against AI that infer emotions. - [Google News](https://news.google.com/read/CBMimAFBVV95cUxQV1BmZFhxMDNPWU1PUHY1QnZiLWlEd0gzeWF4aU9DR3BKc3gxWjhfRy1GWlRLdVpVME9hc1o4RWFNTldCV2dUZGRVcFZtOE5OME9wai11U0pYdFVUbWRtVHVvXzRhSHZUQ2RqMzNqRzFGSE9yYTZ3akQxRkl2WnU0SFA3OWRBbHA5blFCZTRNMmlvaFA4cmtpaw)
- OpenAI releases Advanced Voice mode for paid U.S. subscribers, offering conversational interactions with new voices. - [ExtremeTech](https://www.extremetech.com/internet/openai-releases-chatgpt-advanced-voice-mode-for-paid-subscribers)
- EU ChatGPT Plus users can't access Advanced Voice mode due to regulatory issues, causing frustration. - [BGR](https://bgr.com/tech/im-a-chatgpt-plus-user-but-no-advanced-voice-mode/)

## Meta's Magical Makeover: From Metaverse to Multimodal Marvels

- Meta shifts focus to AI and AR innovations, unveiling the Llama 3.2 AI model and AI-generated avatars at a developer event. - [The Drum](https://www.thedrum.com/news/2024/09/25/meta-emphasizes-ai-and-ar-during-developer-event-signaling-shift-business-priorities)
- Llama 3.2 Vision AI model is now accessible via Hugging Face, enabling image interaction. - [VentureBeat](https://venturebeat.com/ai/heres-how-to-try-metas-new-llama-3-2-with-vision-for-free/)
- Mark Zuckerberg announces Meta AI nearing 500 million users, set to be the most utilized AI assistant. - [Google News](https://news.google.com/read/CBMilwFBVV95cUxNX3VoN0I5WG04WWgxV21yb1dPQkN6OHVJd2FONGYyZk1DV3l0UEEtaHpncTJMV1h3NzZjOEQ1LWp3cFZzQUFrci1XcVJ3UFVfVXFVdUVhWncweTBsTEtDS2pnTF9Ca3lKX2p1RTFSYXgwRGM2b1JPRlZjcnNMQ2dUa1I1eHZ6aWlkS1h6V0JYamY2QlZhT3JV)
- Meta introduces AI-generated posts tailored to user interests on Facebook and Instagram. - [Axios](https://www.axios.com/2024/09/27/meta-ai-posts-facebook-instagram)

## Fintech Figures Out: GPT Revolutionizes Mortgages

- Figure launches an AI tool powered by GPT-4, reducing the mortgage lending process from 45 days to just five. - [TechCrunch](https://techcrunch.com/2024/09/26/former-brex-coo-who-now-heads-unicorn-fintech-figure-says-gpt-is-already-upending-the-mortgage-industry/)

## Hugging Face Hits a Million: Model Mania

- Hugging Face surpasses 1 million AI models, driven by community customization and the demand for specialized models. - [Ars Technica](https://arstechnica.com/information-technology/2024/09/ai-hosting-platform-surpasses-1-million-models-for-the-first-time/)

## Samsung Sprinkles AI on Smartphones and Slates

- Samsung unveils the Galaxy S24 FE smartphone and Galaxy S10 tablets, bringing flagship AI capabilities and AI-driven productivity features to mid-range devices. - [Mobile World Live](https://www.mobileworldlive.com/samsung/samsung-trickles-latest-ai-functions-to-mid-range/), [Google News](https://news.google.com/read/CBMimAFBVV95cUxPUW5DbW5ycldCak5uT0VlUEluYTQ0UDRINDZQdWNDVXZpM3E3a3NWTTF1TXA2Y0xoVXZjWi1xcExFMVpwLWJhcTVtZDdvMnIxZUw5eUl1b1BQd1poLXZHc1J4aEctTEk4QjhtOWpRV2oyWDdsRFktYzc1bFBwVl8wdC0tY2xPS2tNVmVHZ19GQVZuZGoyUzdzcA)

## AI Ethics and Edicts: Regulations Rising

- California's SB 1047 seeks to regulate powerful AI systems by requiring safety protocols; awaiting Governor Newsom's signature. - [The Bulletin](https://thebulletin.org/2024/09/why-california-gov-gavin-newsom-should-sign-an-ai-model-regulation-bill/)
- FTC's "Operation AI Comply" targets companies for misleading AI claims, expanding AI oversight. - [Bloomberg Law](https://news.bloomberglaw.com/privacy-and-data-security/ftc-enforcement-sweep-stretches-boundaries-on-ai-oversight)
- Whistleblower warns OpenAI's shift to for-profit may compromise safety and governance. - [Google News](https://news.google.com/read/CBMiywFBVV95cUxPR21HOEU1ZmVxMDB2VnNFM2dBVmM0ZHVWYUZ6cU10QUo3RUJyV3k4ejNXcjRLYVYzZUx6c0huZlZTblBxVEpVbG9lb29XdVdZbTFIeUxGZExYSWpkT0FPMlUyS285bVZrWmFNYmhxekVNTnVIY3JFSnlrdlR0R1N3aG5QUVJfMnRrbVJCSWV0dEJQRi0xWnlvNDlQSzczZl9GTTY5VDBNcHBlUkdQb21maW95V3JFTEt2bnVBcWdTYjlaX0M4YzBQUkxHd9IBywFBVV95cUxQSWxiMzNSTXJOYVBZTEk2YzQzWW4tcnlEYkVtbmZRUVRwM3NLUzBFWHo1cTlveFlsY3UtYjBLVE1INTBwcjdXd2h5TXlSU3RwblhMUzZlcU9BZk9kM2RvREYzdU5ScUQtUnpUejhkN1h5dWR0SzZqRVZhT0hJYmNhY2c3LVJwTUdzN3pmWnh4aWVBYWFjbnB1UkY2UTlLNDl5UkxmS2U5ZzZ3VmF4dW9HSDZ3QUo2ZjR0SXpLWG5zckxzR1dBUEpXWjZ4RQ)

## The AI Talent Tussle: Star Quarterbacks of Tech

- Google rehires AI expert Noam Shazeer for a reported \\$2.7 billion, highlighting fierce competition for AI talent. - [Business Insider](https://www.businessinsider.com/google-noam-shazeer-deal-ai-talent-hiring-star-quarterbacks-i2024-9)
- AI experts like Shazeer are considered the "star quarterbacks" of tech due to their unique skills. - [Mashable](https://in.mashable.com/tech/82944/sundar-pichai-led-google-rehires-its-top-ai-talent-for-a-whopping-27-billion-paycheck)

## AI's Energy Appetite: Nuclear is the New Neural

- AI data centers' energy demands drive surge in utility stocks, making nuclear power a new AI trade. - [Google News](https://news.google.com/read/CBMifEFVX3lxTE1MNWI3SGxwY0R3ZVJOczdzei1ZZmZ3QWVqaERkTWNpcmFubWdGQlFSVnAzOGlpYWFNZkxhVzhlS2REUVNxM0dGX01ZTVlaVkpycXVmZmpJSTFqbXB1eWhfLUI2Q3pWSHNLMjN1eE9kRVBtcHotSGhoc1Q1OWs)
- Constellation Energy's stock rises after supplying nuclear power to Microsoft's AI data centers. - [Business Insider](https://www.businessinsider.com/one-chipmaker-finally-reaping-the-benefits-of-ai-chip-boom-2024-9)

## Deepfake Dangers: Digital Deceptions on the Rise

- Neo-Nazis use AI to rebrand Hitler, creating AI-edited videos to spread misinformation online. - [The Washington Post](https://www.washingtonpost.com/technology/2024/09/27/neonazis-ai-hitler-videos/)
- AI-generated videos are becoming more realistic, making it crucial to learn how to spot fakes as they improve. - [CNET](https://www.cnet.com/tech/services-and-software/this-ai-generated-video-was-wild-how-to-spot-fakes-as-they-keep-getting-better/#ftag=CAD590a51e)


In [ ]:
def fn_is_revision_complete(state: AgentState) -> str:
    """update edit_complete if MAX_EDITS exceeded"
    return "complete" if edit_complete else "incomplete"
    """

    if state["n_edits"] >= MAX_EDITS:
        log("Max edits reached")
        state["edit_complete"] = True


    return "complete" if state["edit_complete"] else "incomplete"


if DEBUG:
    fn_is_revision_complete(test_state)


In [ ]:
def fn_send_mail(state: AgentState) -> AgentState:

    log("Sending summary email")
    # Convert Markdown to HTML
    html_str = markdown.markdown(state['summary'], extensions=['extra'])
    # send email
    subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
    send_gmail(subject, html_str)
    return state


if DEBUG:
    fn_send_mail(test_state)

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        self.graph = graph
        graph.add_node("initialize", self.initialize)
        graph.add_node("download_sources", self.download_sources)
        graph.add_node("extract_urls", self.extract_urls)
        graph.add_node("filter_urls", self.filter_urls)
        graph.add_node("topic_analysis", self.topic_analysis)
        graph.add_node("topic_clusters", self.topic_clusters)
        graph.add_node("download_pages", self.download_pages)
        graph.add_node("summarize_pages", self.summarize_pages)
        graph.add_node("propose_topics", self.propose_topics)
        graph.add_node("compose_summary", self.compose_summary)
        graph.add_node("rewrite_summary", self.rewrite_summary)
        graph.add_node("send_mail", self.send_mail)

        graph.add_edge(START, "initialize")
        graph.add_edge("initialize", "download_sources")
        graph.add_edge("download_sources", "extract_urls")
        graph.add_edge("extract_urls", "filter_urls")
        graph.add_edge("filter_urls", "topic_analysis")
        graph.add_edge("topic_analysis", "topic_clusters")
        graph.add_edge("topic_clusters", "download_pages")
        graph.add_edge("download_pages", "summarize_pages")
        graph.add_edge("summarize_pages", "propose_topics")
        graph.add_edge("propose_topics", "compose_summary")
        graph.add_edge("compose_summary", "rewrite_summary")
        graph.add_conditional_edges("rewrite_summary",
                                    self.is_revision_complete,
                                    {"incomplete": "rewrite_summary",
                                     "complete": "send_mail",
                                    })
        graph.add_edge("send_mail", END)

        checkpointer = SqliteSaver.from_conn_string(":memory:")
        app = graph.compile(checkpointer=checkpointer, interrupt_before=["compose_summary",])
        self.app = app

#         self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def initialize(self, state: AgentState) -> AgentState:
        self.state = fn_initialize(state)
        return self.state

    def download_sources(self, state: AgentState) -> AgentState:
        self.state = fn_download_sources(state)
        return self.state

    def extract_urls(self, state: AgentState) -> AgentState:
        self.state = fn_extract_urls(state)
        return self.state

    def filter_urls(self, state: AgentState) -> AgentState:
        self.state = fn_filter_urls(state)
        return self.state

    def topic_analysis(self, state: AgentState) -> AgentState:
        self.state = fn_topic_analysis(state)
        return self.state

    def topic_clusters(self, state: AgentState) -> AgentState:
        self.state = fn_topic_clusters(state)
        return self.state

    def download_pages(self, state: AgentState) -> AgentState:
        self.state = fn_download_pages(state)
        return self.state

    def summarize_pages(self, state: AgentState) -> AgentState:
        self.state = fn_summarize_pages(state)
        return self.state

    def propose_topics(self, state: AgentState) -> AgentState:
        self.state = fn_propose_cats(state)
        return self.state

    def compose_summary(self, state: AgentState) -> AgentState:
        self.state = fn_compose_summary(state)
        return self.state

    def rewrite_summary(self, state: AgentState) -> AgentState:
        self.state = fn_rewrite_summary(state)
        return self.state

    def is_revision_complete(self, state: AgentState) -> str:
        return fn_is_revision_complete(state)

    def send_mail(self, state: AgentState) -> AgentState:
        self.state = fn_send_mail(state)
        return self.state

    def run(self, config, state):
        self.state = state
        for step in self.app.stream(state, config, stream_mode="debug"):
            if step["type"] == "checkpoint":
                display(f'Step {step["step"]}')

            # Check if there is an error message in the state
#             if step["payload"].get("values") and step["payload"]["values"].get("error_message"):
#                 # Update the error state to 'tested' since error simulation is complete
#                 self.state['error_state'] = 'tested'
#                 # Retry from the last checkpoint saved by the checkpointer
#                 for retry_step in graph.stream(self.state, config, stream_mode="debug"):
#                     if retry_step["type"] == "checkpoint":
#                         print(retry_step["step"], retry_step["payload"].get("values"))
#                 break
        return None


In [ ]:
model = ChatOpenAI(model=MODEL)
lg_agent = Agent(model, [], system="")


In [ ]:
# Image(lg_agent.graph.get_graph().draw_png())
Image(lg_agent.app.get_graph().draw_mermaid_png())


In [ ]:
# Configuration with thread ID for checkpointing
# Generate a random UUID
thread_id = uuid.uuid4()

config = {"configurable": {"thread_id": thread_id}}
# initial state
state = AgentState()
do_download = True
state["do_download"] = do_download
# before_date="2024-07-22 10:00:00"
# state["before_date"] = before_date
log(f"Starting with before_date={state.get('before_date')}, do_download={do_download}, thread_id={thread_id}")
lg_agent.run(config, state)


In [ ]:
user_approval = input("Edit topics? (yes/no): ")


In [ ]:
if user_approval.lower() == "yes":
    # If approved, continue the graph execution
    for step in lg_agent.graph.stream(None, config, stream_mode="debug"):
        if step["type"] == "checkpoint":
            display(f'Step {step["step"]}')
else:
    print("Operation cancelled by user.")


In [ ]:
# get last state
last_state = lg_agent.graph.get_state(config)
pd.DataFrame(last_state.values['AIdf'])


In [ ]:
# update state as node
# lg_agent.graph.update_state(config, {"foo": 2, "bar": ["b"]}, as_node="revise_summary")


In [ ]:
# resume from error starting at a node
# result = lg_agent.graph.invoke(None, {'configurable': {'thread_id': 'newsbot-thread-5'}, "next": "topic_analysis"})